# Sample Notebook - AIDC with WML

In [5]:
## Import AIDC libraries
!pip install /project_data/data_asset/aidc-1.3.zip --force-reinstall;
import aidc

Processing /project_data/data_asset/aidc-1.3.zip
  Preparing metadata (setup.py) ... done
  Using cached javascript-1%211.0.3-py3-none-any.whl (33 kB)
  Created wheel for aidc: filename=aidc-1.3-py3-none-any.whl size=138109 sha256=848ef62e87b6c702a4fcd1decab0297d9a053cd243be6a8755f4e090978407c5
  Stored in directory: /tmp/1000710000/.cache/pip/wheels/36/35/e2/ffafef2711229494b6f5514e8d265d330c9b11bfa3285f4fce
Successfully built aidc
  Attempting uninstall: javascript
    Found existing installation: javascript 1!1.0.3
    Uninstalling javascript-1!1.0.3:
      Successfully uninstalled javascript-1!1.0.3
  Attempting uninstall: aidc
    Found existing installation: aidc 1.3
    Uninstalling aidc-1.3:
      Successfully uninstalled aidc-1.3


In [6]:
## Load the AIDC model from JSON
import json
read_file = open('/project_data/data_asset/AIDC_model.json')
model_json = json.load(read_file)
load_model =aidc.load_task_model(model_json)
print(aidc.dispatch_description(load_model))

if confidence <27.917%: use ml else 
if confidence <55.17%: use human else 
use ml 


In [7]:
#Test the dispatch on the model
case_for_ml={'case': {'caseId': 'CaseDataModel0', 'CheckingStatus': 'no_checking', 'CreditHistory': 'prior_payments_delayed', 'LoanPurpose': 'education', 'ExistingSavings': '100_to_500', 'EmploymentDuration': '1_to_4', 'InstallmentPercent': '3', 'Sex': 'male', 'OthersOnLoan': 'co-applicant', 
               'CurrentResidenceDuration': '2', 'OwnsProperty': 'real_estate', 'InstallmentPlans': 'bank', 'Housing': 'rent', 'ExistingCreditsCount': '1', 'Job': 'skilled', 'Dependents': '1', 'Telephone': 'yes', 'ForeignWorker': 'no', 'Age': '31', 'LoanAmount': '2000', 'LoanDuration': '34'},
      'decisionTools': {'ml': {'outcome': 'Risk', 'confidence': 0.27}}}
dispatch_result = aidc.dispatch(load_model, case_for_ml)
print(dispatch_result)

case_for_human={'case': {'caseId': 'CaseDataModel0', 'CheckingStatus': 'no_checking', 'CreditHistory': 'prior_payments_delayed', 'LoanPurpose': 'education', 'ExistingSavings': '100_to_500', 'EmploymentDuration': '1_to_4', 'InstallmentPercent': '3', 'Sex': 'male', 'OthersOnLoan': 'co-applicant', 
               'CurrentResidenceDuration': '2', 'OwnsProperty': 'real_estate', 'InstallmentPlans': 'bank', 'Housing': 'rent', 'ExistingCreditsCount': '1', 'Job': 'skilled', 'Dependents': '1', 'Telephone': 'yes', 'ForeignWorker': 'no', 'Age': '31', 'LoanAmount': '2000', 'LoanDuration': '34'},
      'decisionTools': {'ml': {'outcome': 'Risk', 'confidence': 0.28}}}
dispatch_result = aidc.dispatch(load_model, case_for_human)
print(dispatch_result)

{
  toolToUse: 'ml',
  caseId: undefined,
  type: 'ml',
  confidence: 0.27,
  outcome: 'Risk'
}
{ toolToUse: 'human', caseId: undefined, type: 'h' }


In [8]:
#Configure the WML client
import os
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "instance_id": "openshift",
    "url": os.environ.get("RUNTIME_ENV_APSX_URL"),
    "version": "4.7"
}
client = APIClient(wml_credentials)

metadata = {            
     client.spaces.ConfigurationMetaNames.NAME: 'AIDC_space',         
     client.spaces.ConfigurationMetaNames.DESCRIPTION:  'AIDC space',            
     }
SPACE_NAME=metadata[client.spaces.ConfigurationMetaNames.NAME]

def guid_from_space_name(client, space_name):
 space = client.spaces.get_details()
 try:
     return_item=next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id']
 except:
     return_item="0"
 return(return_item)

space_uid = guid_from_space_name(client, SPACE_NAME)
if(space_uid=="0"):
    space_details = client.spaces.store(meta_props=metadata,background_mode=False)
    space_uid = guid_from_space_name(client, SPACE_NAME)
client.set.default_space(space_uid)

'SUCCESS'

In [9]:
#Configure the model endpoint
model_serving_url='https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/cd7cded4-d757-477a-a716-d56c7ba3072c/predictions'

In [10]:
#AIDC custom_function
def my_deployable_function():
    import aidc
    import json
    import requests
    from ibm_watson_studio_lib import access_project_or_space    
    wslib = access_project_or_space()
    token = wslib.auth.get_current_token()
    
    default_model={'id': '0', 'name': 'taskModel', 'caseDataModel': {'id': 'CaseDataModel0', 'comments': [], 'attributes': {'CheckingStatus': {'name': 'CheckingStatus', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'0_to_200': {'name': '0_to_200', '_className': 'AttributeValue', '_version': 1}, 'less_0': {'name': 'less_0', '_className': 'AttributeValue', '_version': 1}, 'no_checking': {'name': 'no_checking', '_className': 'AttributeValue', '_version': 1}, 'greater_200': {'name': 'greater_200', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'LoanDuration': {'name': 'LoanDuration', 'isUsable': True, 'type': 'continuous', 'range': [4, 64], 'values': {}, '_className': 'Attribute', '_version': 1}, 'CreditHistory': {'name': 'CreditHistory', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'credits_paid_to_date': {'name': 'credits_paid_to_date', '_className': 'AttributeValue', '_version': 1}, 'prior_payments_delayed': {'name': 'prior_payments_delayed', '_className': 'AttributeValue', '_version': 1}, 'outstanding_credit': {'name': 'outstanding_credit', '_className': 'AttributeValue', '_version': 1}, 'all_credits_paid_back': {'name': 'all_credits_paid_back', '_className': 'AttributeValue', '_version': 1}, 'no_credits': {'name': 'no_credits', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'LoanPurpose': {'name': 'LoanPurpose', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'other': {'name': 'other', '_className': 'AttributeValue', '_version': 1}, 'car_new': {'name': 'car_new', '_className': 'AttributeValue', '_version': 1}, 'furniture': {'name': 'furniture', '_className': 'AttributeValue', '_version': 1}, 'retraining': {'name': 'retraining', '_className': 'AttributeValue', '_version': 1}, 'education': {'name': 'education', '_className': 'AttributeValue', '_version': 1}, 'vacation': {'name': 'vacation', '_className': 'AttributeValue', '_version': 1}, 'appliances': {'name': 'appliances', '_className': 'AttributeValue', '_version': 1}, 'car_used': {'name': 'car_used', '_className': 'AttributeValue', '_version': 1}, 'repairs': {'name': 'repairs', '_className': 'AttributeValue', '_version': 1}, 'radio_tv': {'name': 'radio_tv', '_className': 'AttributeValue', '_version': 1}, 'business': {'name': 'business', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'LoanAmount': {'name': 'LoanAmount', 'isUsable': True, 'type': 'continuous', 'range': [250, 11676], 'values': {}, '_className': 'Attribute', '_version': 1}, 'ExistingSavings': {'name': 'ExistingSavings', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'100_to_500': {'name': '100_to_500', '_className': 'AttributeValue', '_version': 1}, 'less_100': {'name': 'less_100', '_className': 'AttributeValue', '_version': 1}, '500_to_1000': {'name': '500_to_1000', '_className': 'AttributeValue', '_version': 1}, 'unknown': {'name': 'unknown', '_className': 'AttributeValue', '_version': 1}, 'greater_1000': {'name': 'greater_1000', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'EmploymentDuration': {'name': 'EmploymentDuration', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'less_1': {'name': 'less_1', '_className': 'AttributeValue', '_version': 1}, '1_to_4': {'name': '1_to_4', '_className': 'AttributeValue', '_version': 1}, 'greater_7': {'name': 'greater_7', '_className': 'AttributeValue', '_version': 1}, '4_to_7': {'name': '4_to_7', '_className': 'AttributeValue', '_version': 1}, 'unemployed': {'name': 'unemployed', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'InstallmentPercent': {'name': 'InstallmentPercent', 'isUsable': True, 'type': 'discrete', 'range': [1, 6], 'values': {'1': {'name': '1', '_className': 'AttributeValue', '_version': 1}, '2': {'name': '2', '_className': 'AttributeValue', '_version': 1}, '3': {'name': '3', '_className': 'AttributeValue', '_version': 1}, '4': {'name': '4', '_className': 'AttributeValue', '_version': 1}, '5': {'name': '5', '_className': 'AttributeValue', '_version': 1}, '6': {'name': '6', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Sex': {'name': 'Sex', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'female': {'name': 'female', '_className': 'AttributeValue', '_version': 1}, 'male': {'name': 'male', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'OthersOnLoan': {'name': 'OthersOnLoan', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'none': {'name': 'none', '_className': 'AttributeValue', '_version': 1}, 'co-applicant': {'name': 'co-applicant', '_className': 'AttributeValue', '_version': 1}, 'guarantor': {'name': 'guarantor', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'CurrentResidenceDuration': {'name': 'CurrentResidenceDuration', 'isUsable': True, 'type': 'discrete', 'range': [1, 6], 'values': {'1': {'name': '1', '_className': 'AttributeValue', '_version': 1}, '2': {'name': '2', '_className': 'AttributeValue', '_version': 1}, '3': {'name': '3', '_className': 'AttributeValue', '_version': 1}, '4': {'name': '4', '_className': 'AttributeValue', '_version': 1}, '5': {'name': '5', '_className': 'AttributeValue', '_version': 1}, '6': {'name': '6', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'OwnsProperty': {'name': 'OwnsProperty', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'savings_insurance': {'name': 'savings_insurance', '_className': 'AttributeValue', '_version': 1}, 'real_estate': {'name': 'real_estate', '_className': 'AttributeValue', '_version': 1}, 'unknown': {'name': 'unknown', '_className': 'AttributeValue', '_version': 1}, 'car_other': {'name': 'car_other', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Age': {'name': 'Age', 'isUsable': True, 'type': 'continuous', 'range': [19, 74], 'values': {}, '_className': 'Attribute', '_version': 1}, 'InstallmentPlans': {'name': 'InstallmentPlans', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'none': {'name': 'none', '_className': 'AttributeValue', '_version': 1}, 'stores': {'name': 'stores', '_className': 'AttributeValue', '_version': 1}, 'bank': {'name': 'bank', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Housing': {'name': 'Housing', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'own': {'name': 'own', '_className': 'AttributeValue', '_version': 1}, 'free': {'name': 'free', '_className': 'AttributeValue', '_version': 1}, 'rent': {'name': 'rent', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'ExistingCreditsCount': {'name': 'ExistingCreditsCount', 'isUsable': True, 'type': 'discrete', 'range': [1, 4], 'values': {'1': {'name': '1', '_className': 'AttributeValue', '_version': 1}, '2': {'name': '2', '_className': 'AttributeValue', '_version': 1}, '3': {'name': '3', '_className': 'AttributeValue', '_version': 1}, '4': {'name': '4', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Job': {'name': 'Job', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'skilled': {'name': 'skilled', '_className': 'AttributeValue', '_version': 1}, 'management_self-employed': {'name': 'management_self-employed', '_className': 'AttributeValue', '_version': 1}, 'unskilled': {'name': 'unskilled', '_className': 'AttributeValue', '_version': 1}, 'unemployed': {'name': 'unemployed', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Dependents': {'name': 'Dependents', 'isUsable': True, 'type': 'discrete', 'range': [1, 2], 'values': {'1': {'name': '1', '_className': 'AttributeValue', '_version': 1}, '2': {'name': '2', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Telephone': {'name': 'Telephone', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'none': {'name': 'none', '_className': 'AttributeValue', '_version': 1}, 'yes': {'name': 'yes', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'ForeignWorker': {'name': 'ForeignWorker', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'yes': {'name': 'yes', '_className': 'AttributeValue', '_version': 1}, 'no': {'name': 'no', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}}, 'outcomes': ['No Risk', 'Risk'], 'targetOutcome': 'No Risk', '_className': 'CaseDataModel', '_version': 1}, 'decisionMetrics': {'Scenario0': {'id': 'DecisionMetrics2', 'comments': [], 'costMatrix': {'No Risk': {'No Risk': {'value': '0', '_className': 'Expression', '_version': 1}, 'Risk': {'value': '-LoanAmount*0.01', '_className': 'Expression', '_version': 1}}, 'Risk': {'No Risk': {'value': '-LoanAmount*0.5', '_className': 'Expression', '_version': 1}, 'Risk': {'value': 'LoanAmount*0.3', '_className': 'Expression', '_version': 1}}}, 'toolCosts': {'DecisionTool1': {'value': '-0.02', '_className': 'Expression', '_version': 1}, 'DecisionTool2': {'value': '-0.05', '_className': 'Expression', '_version': 1}}, 'indicators': {'Performance': {'name': 'Performance', 'expression': '0', 'aggregation': 'Average', '_className': 'PerformanceIndicator', '_version': 1}, 'Impact': {'name': 'Impact', 'expression': '0', 'aggregation': 'Average', '_className': 'ImpactIndicator', '_version': 1}, 'Decision Cost': {'name': 'Decision Cost', 'expression': '0', 'aggregation': 'Average', '_className': 'DecisionCostIndicator', '_version': 1}, 'human volume': {'name': 'human volume', 'expression': '0', 'aggregation': '%', 'tool': 'DecisionTool1', '_className': 'ToolUsageIndicator', '_version': 1}, 'ml volume': {'name': 'ml volume', 'expression': '0', 'aggregation': '%', 'tool': 'DecisionTool2', '_className': 'ToolUsageIndicator', '_version': 1}}, 'commentMatrix': {'No Risk': {'No Risk': '', 'Risk': ''}, 'Risk': {'No Risk': '', 'Risk': ''}}, 'windowWidth': 0.1, 'samples': 100, 'confidenceIntervalWidth': 1.96, 'robustness': 0.5, '_className': 'DecisionMetrics', '_version': 1}}, 'defaultMetrics': 'DecisionMetrics2', 'decisionTools': {'DecisionTool1': {'id': 'DecisionTool1', 'name': 'human', 'comments': [], 'type': 'h', 'hasHuman': True, '_className': 'DecisionTool', '_version': 1}, 'DecisionTool2': {'id': 'DecisionTool2', 'name': 'ml', 'comments': [], 'type': 'ml', 'hasHuman': False, '_className': 'DecisionTool', '_version': 1}}, 'caseSources': {}, 'analyses': {}, 'scenarios': {'Scenario0': {'id': 'Scenario0', 'name': 'initial', 'description': 'simple scenario: uniform costs', 'comments': [], 'decisionTools': ['DecisionTool1', 'DecisionTool2'], 'decisionMetrics': {'id': 'DecisionMetrics2', 'comments': [], 'costMatrix': {'No Risk': {'No Risk': {'value': '0', '_className': 'Expression', '_version': 1}, 'Risk': {'value': '-LoanAmount*0.01', '_className': 'Expression', '_version': 1}}, 'Risk': {'No Risk': {'value': '-LoanAmount*0.5', '_className': 'Expression', '_version': 1}, 'Risk': {'value': 'LoanAmount*0.3', '_className': 'Expression', '_version': 1}}}, 'toolCosts': {'DecisionTool1': {'value': '-0.02', '_className': 'Expression', '_version': 1}, 'DecisionTool2': {'value': '-0.05', '_className': 'Expression', '_version': 1}}, 'indicators': {'Performance': {'name': 'Performance', 'expression': '0', 'aggregation': 'Average', '_className': 'PerformanceIndicator', '_version': 1}, 'Impact': {'name': 'Impact', 'expression': '0', 'aggregation': 'Average', '_className': 'ImpactIndicator', '_version': 1}, 'Decision Cost': {'name': 'Decision Cost', 'expression': '0', 'aggregation': 'Average', '_className': 'DecisionCostIndicator', '_version': 1}, 'human volume': {'name': 'human volume', 'expression': '0', 'aggregation': '%', 'tool': 'DecisionTool1', '_className': 'ToolUsageIndicator', '_version': 1}, 'ml volume': {'name': 'ml volume', 'expression': '0', 'aggregation': '%', 'tool': 'DecisionTool2', '_className': 'ToolUsageIndicator', '_version': 1}}, 'commentMatrix': {'No Risk': {'No Risk': '', 'Risk': ''}, 'Risk': {'No Risk': '', 'Risk': ''}}, 'windowWidth': 0.1, 'samples': 100, 'confidenceIntervalWidth': 1.96, 'robustness': 0.5, '_className': 'DecisionMetrics', '_version': 1}, 'caseSources': ['TableCaseSource0'], 'selectedAttributes': [], 'computedTools': {'CompositeDecisionTool0': {'id': 'CompositeDecisionTool0', 'name': 'initial scenario', 'comments': [], 'type': 'o', 'hasHuman': False, 'tools': ['DecisionTool1', 'DecisionTool2'], 'confidenceProvider': 'DecisionTool2', 'attributes': [], 'useDeciles': False, 'dispatchConfiguration': {'tools': ['DecisionTool1', 'DecisionTool2'], 'useDeciles': False, 'decisionTree': {'bestTool': 'DecisionTool2', 'basisCount': 4961, 'basisPerf': 0.8330981656924007, 'basisStdDev': 0.5531251633400509, 'basisConfidence95': 0.01539199360054257, 'secondTool': 'DecisionTool1', 'secondPerf': 0.5895988711953235, 'secondStdDev': 0.8076962121275549, 'secondConfidence95': 0.022476024871439102, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.5895988711953235, 0.5895988711953235, 0, 4961, 0.8076962121275549, 0.022476024871439102]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.8330981656924007, 0.8330981656924007, 0, 4961, 0.5531251633400509, 0.01539199360054257]], '_className': 'AnalysisResult', '_version': 1}}, 'slots': [{'bestTool': 'DecisionTool2', 'basisCount': 1228, 'basisPerf': 0.8892508143322475, 'basisStdDev': 0.4574199265548392, 'basisConfidence95': 0.025584207442834406, 'secondTool': 'DecisionTool1', 'secondPerf': 0.6042345276872965, 'secondStdDev': 0.7968065232856156, 'secondConfidence95': 0.044566627293834905, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.6042345276872965, 0.6042345276872965, 0, 1228, 0.7968065232856156, 0.044566627293834905]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.8892508143322475, 0.8892508143322475, 0, 1228, 0.4574199265548392, 0.025584207442834406]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 0, 'lowBound': 0, 'highBound': 0.35420247908907687, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool1', 'basisCount': 807, 'basisPerf': 0.6158612143742255, 'basisStdDev': 0.7878546595848148, 'basisConfidence95': 0.05435824314143555, 'secondTool': 'DecisionTool2', 'secondPerf': 0.34820322180916974, 'secondStdDev': 0.9374190718785885, 'secondConfidence95': 0.06467747980502939, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.6158612143742255, 0.6158612143742255, 0, 807, 0.7878546595848148, 0.05435824314143555]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.34820322180916974, 0.34820322180916974, 0, 807, 0.9374190718785885, 0.06467747980502939]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 1, 'lowBound': 0.35420247908907687, 'highBound': 0.5946194015953628, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool2', 'basisCount': 2926, 'basisPerf': 0.9432672590567327, 'basisStdDev': 0.33203445301895856, 'basisConfidence95': 0.01203100924085604, 'secondTool': 'DecisionTool1', 'secondPerf': 0.5762132604237867, 'secondStdDev': 0.8172993812011535, 'secondConfidence95': 0.029614205147607257, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.5762132604237867, 0.5762132604237867, 0, 2926, 0.8172993812011535, 0.029614205147607257]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.9432672590567327, 0.9432672590567327, 0, 2926, 0.33203445301895856, 0.01203100924085604]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 2, 'lowBound': 0.5946194015953628, 'highBound': 1, '_className': 'DecisionNode', '_version': 1}], 'confidenceProvider': 'DecisionTool2', 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 174.6179886517464, 0.5983699345819098, 0.8012199581815476, 0.07262018388480501], [101, 504, 666.3114308957505, 198.2302479314591, 0.5950077958739799, 0.8037199281150049, 0.06807220331597795], [65, 551, 721.1424425150965, 215.28796056247091, 0.5970747188630879, 0.8021856269558587, 0.06537067678413065], [53, 593, 757.4765311869324, 226.43260299724597, 0.5978603789676125, 0.8016002540298396, 0.06376030760336958], [54, 620, 780.4182089417114, 232.56188996923282, 0.595992987617752, 0.8029896379844926, 0.06287056234648003], [39, 648, 787.2394933018742, 233.76482829427698, 0.5938849112201177, 0.8045500060437968, 0.06265837910137374], [32, 674, 783.295106379799, 232.20350609091022, 0.5928889487490834, 0.8052842320890218, 0.06284459943478773], [31, 705, 772.3184664651072, 228.25290893130435, 0.5910849444686084, 0.8066093158540518, 0.06334166551119995], [34, 743, 755.5948864772723, 222.9925030498541, 0.5902435472783246, 0.8072252194377361, 0.0640632441017021], [25, 770, 733.2801280101717, 217.73768708306162, 0.5938731427890032, 0.8045586928707631, 0.06492320921531645], [23, 798, 705.0823953200298, 211.2340458580027, 0.5991754928503801, 0.8006177169957611, 0.06604633666191896], [46, 731, 679.6849616043089, 205.1316032204266, 0.6036078913272992, 0.7972813264635075, 0.06712866911980782], [38, 693, 661.2234869938007, 199.12191095482112, 0.6022832366711982, 0.7982824705734598, 0.06810205733332392], [40, 665, 645.7656636747749, 194.50028298542458, 0.6023865743452727, 0.798204494505366, 0.06890895774016959], [24, 639, 631.3836409147973, 190.66550849798242, 0.6039608762169749, 0.7970139647454272, 0.06963737216256732], [27, 618, 623.6519062582148, 188.36701854482294, 0.6040774241354826, 0.7969256337010636, 0.07006382490416481], [26, 607, 618.1991892236267, 186.0279310694293, 0.6018381593254912, 0.7986180751646587, 0.07044682483825361], [37, 605, 613.1480790457492, 182.9179053144092, 0.5966516460398501, 0.8025003509519089, 0.07090812487672805], [40, 598, 605.9192816957346, 179.1082243974069, 0.591194998436597, 0.8065286565420672, 0.07150865117626005], [21, 593, 594.614044958479, 173.50207717010207, 0.5835788059201241, 0.812056511137521, 0.07243218997779463], [26, 605, 585.7606828381125, 168.430191908133, 0.5750819296783778, 0.8180958221121736, 0.07324838435208303], [29, 602, 577.5147629228793, 164.68497962835448, 0.5703230123325742, 0.8214205144771446, 0.07391920689637516], [33, 583, 571.6406726458698, 163.14051806261992, 0.5707799527542895, 0.8211030663283451, 0.07428366923713392], [21, 570, 567.3196075776277, 162.6366525648949, 0.5733510719269153, 0.8193097999659574, 0.0744845587356107], [35, 579, 566.3049097540636, 162.80741878890205, 0.5749814843018277, 0.8181664211577407, 0.07449922134831848], [26, 583, 564.4748875421996, 162.66066276946427, 0.5763255952012708, 0.8172201712610258, 0.0745767232350182], [23, 583, 563.6774808520777, 164.179498961569, 0.5825299201713314, 0.8128092593623555, 0.07442777740235865], [31, 580, 564.080332543945, 168.35667790164302, 0.5969244740101876, 0.802297433828415, 0.07391852531410932], [22, 585, 568.4578596050166, 174.5548439335475, 0.6141346838086961, 0.789201235518034, 0.07302991780566141], [31, 585, 577.8133836442377, 181.2535994943734, 0.6273776434571895, 0.7787151549123089, 0.07195344404588444], [30, 587, 583.890447182401, 186.63764768948144, 0.6392899510177391, 0.7689657720131217, 0.07112853943034837], [26, 588, 588.7332941603438, 190.72394424821422, 0.6479128873464725, 0.761714441513588, 0.0705006088682188], [30, 577, 593.3288656184958, 193.4134109853211, 0.6519602270949814, 0.758253165035439, 0.07006731064801819], [35, 593, 599.1144178234287, 195.4516754496951, 0.652468609117328, 0.757815752090505, 0.06970806058466304], [30, 596, 604.0168762582872, 197.55843795041267, 0.6541487356255045, 0.7563659376780232, 0.06935815247425357], [32, 592, 608.7374278541419, 200.19832514147947, 0.6577493545852696, 0.7532368727981, 0.06894564751838878], [27, 601, 612.3128676974197, 201.26051955724134, 0.6573780502574581, 0.7535609458031278, 0.06875884447165989], [42, 616, 615.8680691275756, 200.90633940149814, 0.6524330436098024, 0.7578463720346285, 0.06875476711693544], [34, 627, 618.3707451800456, 200.0727799048843, 0.6470965240977914, 0.7624080852801588, 0.06882169275785256], [20, 629, 618.4372143951659, 199.58574140742445, 0.6454519125360854, 0.763800909009351, 0.06888082643247201], [30, 616, 619.7894474788376, 199.21332399249266, 0.6428419354438741, 0.7659988551132261, 0.06890457252851677], [29, 618, 621.1703968707503, 199.04861607095526, 0.6408824923843632, 0.7676389978076978, 0.06890158477802195], [24, 632, 624.4337280335847, 199.52280884936764, 0.6390519918829127, 0.7691635402633711, 0.0687895137780587], [41, 640, 629.4098427218046, 200.6440211635571, 0.6375623879534422, 0.7703987288846642, 0.06857204240999487], [32, 644, 634.539318869226, 201.14216664399635, 0.6339785752045739, 0.7733506101255604, 0.06842503347790778], [29, 642, 639.7491371710681, 200.8750620824129, 0.6279807206014227, 0.7782288959894235, 0.06836044688173425], [27, 663, 647.2454425303501, 201.5257590147532, 0.6227182016976609, 0.782446190657498, 0.06814732468706892], [47, 669, 656.4827710203692, 203.45455078254386, 0.6198321106472149, 0.7847344484668797, 0.06776505011774134], [35, 664, 662.0507521335019, 203.01151783628617, 0.6132808313624545, 0.7898649390138653, 0.0676997161801749], [22, 680, 670.304518327072, 202.517159612793, 0.6042541981314101, 0.7967916064069477, 0.06757598319415432], [29, 694, 682.0072246014435, 204.36309466223977, 0.5992989144115505, 0.8005253345054976, 0.06715048041872221], [29, 708, 697.0073345999263, 207.54045736883765, 0.5955187185740688, 0.8033414316639594, 0.06654071760974134], [44, 740, 714.6992438595885, 213.2099562178048, 0.5966424563888094, 0.8025071832914181, 0.0656778419562426], [40, 748, 731.6274379687737, 219.20132534290633, 0.599215704516161, 0.8005876213514667, 0.06483589564720069], [33, 766, 749.827523794892, 226.8581185136091, 0.6050941351565108, 0.7961539346126439, 0.06386661596483084], [36, 796, 771.5340044686911, 236.26133898146716, 0.6124456929002529, 0.7905126648244972, 0.06273832960514834], [48, 819, 794.5241232990452, 246.27856162896347, 0.6199397964315011, 0.7846493795323288, 0.061594274660994354], [34, 833, 818.010093419614, 254.78427769319293, 0.6229367577314145, 0.7822722006226939, 0.06061159165424247], [40, 843, 847.193014683848, 265.79685468627605, 0.6274765019998778, 0.7786354984445538, 0.05941990977438823], [40, 860, 874.9118881198643, 276.68029798037, 0.632475799534374, 0.7745801204545303, 0.05831859947803997], [41, 890, 900.7436152744635, 285.0622633842424, 0.6329487293615325, 0.7741937134849518, 0.05746194146515013], [54, 900, 924.6538587617463, 291.94832381904473, 0.6314759216167841, 0.7753954864572227, 0.05675813445563077], [49, 906, 942.2651862480523, 295.1433389753737, 0.6264549370662558, 0.7794576395323316, 0.05637230039816788], [52, 918, 958.0265479843835, 297.42859233149176, 0.6209193115937327, 0.783874485163261, 0.056064836970215505], [49, 933, 972.0301994996937, 297.83684124625347, 0.6128139668902279, 0.7902272090887911, 0.0558846045290085], [55, 948, 982.6950901029222, 297.0511766693579, 0.604564283796811, 0.7965563550416559, 0.05580266389602935], [59, 949, 988.5808782664636, 294.79598391255314, 0.5964023589642877, 0.8026856334965976, 0.055849940330973166], [49, 963, 989.122185648845, 292.57324898591645, 0.5915816129308515, 0.806245121065631, 0.05595831769815825], [49, 976, 986.0002242460553, 289.11961800394073, 0.5864493960435272, 0.8099858677039884, 0.05617670795737217], [47, 994, 984.6273112360902, 286.17572581746316, 0.5812874019474461, 0.8136983202251236, 0.05634454058292994], [43, 1020, 985.5537594129897, 286.37679973004333, 0.5811490179908878, 0.8137971607767064, 0.056321472026459096], [41, 1039, 988.8460923441614, 288.5772646881796, 0.5836646712211351, 0.8119947977470824, 0.05616533375203823], [50, 1029, 997.0593149933917, 293.16984629262885, 0.5880690183303126, 0.8088107502253069, 0.0558237530939084], [53, 1045, 1006.5628540930174, 297.6860316186711, 0.591490199361483, 0.8063121877159689, 0.05547371282035515], [53, 1051, 1017.1405338390672, 303.475513540563, 0.5967228783915107, 0.8024473854429037, 0.05505209792589878], [45, 1048, 1028.7149214037931, 309.65409313117704, 0.6020211949655021, 0.7984801067104357, 0.054606029834043356], [48, 1044, 1042.5732809167782, 317.255481274294, 0.6086008285102378, 0.7934765475656179, 0.05407167505718027], [46, 1059, 1061.073689853978, 326.47994864215104, 0.6153765789576411, 0.7882332561306904, 0.05342083541697771], [62, 1075, 1085.030100296678, 337.12403876203155, 0.6214095602875022, 0.7834859018408025, 0.05266847834592405], [63, 1090, 1106.5207916475836, 347.1526063986797, 0.627466937845384, 0.7786432057822991, 0.05199307831930327], [64, 1151, 1128.466224271798, 354.7871843265215, 0.6287953980288007, 0.777570798974476, 0.051449571282669505], [54, 1156, 1148.5726620458995, 361.1183561946599, 0.628812382755857, 0.7775570636891552, 0.05099680515798133], [59, 1176, 1166.961944162921, 366.29131554410213, 0.6277690842898022, 0.7783996253917035, 0.05062080342341237], [51, 1186, 1185.8114122404581, 371.56885548750427, 0.6266913130570508, 0.7792676036759343, 0.05024485177952995], [60, 1182, 1203.9021559263047, 376.7381143377317, 0.6258616823355755, 0.7799340706650021, 0.04988723358438644], [50, 1187, 1219.252235759979, 381.1839400376302, 0.6252749494447837, 0.7804045345824315, 0.04958715399945932], [54, 1208, 1235.3059949191388, 382.88748833809285, 0.6199071159905705, 0.7846751987569458, 0.049398499798047654], [78, 1247, 1253.5869690944037, 383.89343380218884, 0.6124719596910218, 0.7904923140627239, 0.04921841940044328], [61, 1321, 1273.595021893862, 386.58869087319806, 0.6070826035395973, 0.7946387307950613, 0.04895817939171185], [88, 1420, 1308.5422963048659, 393.83790332536256, 0.6019490610850009, 0.798534487582652, 0.048418243206930285], [70, 1862, 1399.2722750017035, 414.1414376506474, 0.5919383168656625, 0.8059832684530436, 0.04704007114678039], [60, 1804, 1515.6689728755591, 438.8406833933453, 0.5790719362167419, 0.8152764516936528, 0.0454575830046523], [59, 1756, 1619.6537389776506, 460.18787870329703, 0.5682546431112794, 0.8228527575347057, 0.044177999590570866], [64, 1693, 1709.8520717742715, 478.0083384159345, 0.5591224484348719, 0.8290850907212136, 0.04315949501243186], [52, 1635, 1787.62550343858, 490.8267823688385, 0.5491382634950224, 0.8357315164367507, 0.04237904741183825], [56, 1584, 1856.274143641366, 500.85605369380625, 0.539635867266136, 0.841898527591021, 0.0417411960028943], [74, 1532, 1912.3696124229427, 510.6497097556556, 0.5340491779815203, 0.8454534141496276, 0.04121117483429715], [114, 1461, 1947.9050194228269, 514.404769523662, 0.5281620658034779, 0.8491435875316947, 0.04092255585653422], [131, 1398, 1954.9580709104546, 508.09261292260175, 0.5197989874902791, 0.8542886003009074, 0.04097223483980096], [441, 1338, 1920.0845665614802, 489.38020953939207, 0.5097485996835883, 0.8603234072839238, 0.04148840725393848]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 283.87460812585545, 0.9727636425391536, 0.2317992574491087, 0.039060471828444944], [101, 504, 666.3114308957505, 323.7202668253279, 0.9716785629510726, 0.23630651768696073, 0.03691095565380361], [65, 551, 721.1424425150965, 349.8818146442225, 0.9703542435360072, 0.24168707464749345, 0.035881637606271916], [53, 593, 757.4765311869324, 367.11814267843397, 0.969318856923992, 0.24580674037048952, 0.035307614352406175], [54, 620, 780.4182089417114, 377.9132708296802, 0.968489116475513, 0.24905587981109759, 0.035013923132734694], [39, 648, 787.2394933018742, 380.32391300976167, 0.9662216294931814, 0.25771255828837963, 0.03546258794805528], [32, 674, 783.295106379799, 377.3344362673341, 0.9634540882331893, 0.2678735146795123, 0.03624584698305137], [31, 705, 772.3184664651072, 371.14945195312924, 0.961130590731246, 0.2760941643001635, 0.03705838384853375], [34, 743, 755.5948864772723, 361.935335560837, 0.9580142535062636, 0.2867205784014045, 0.03818044609496769], [25, 770, 733.2801280101717, 349.44216424675443, 0.9530932338095685, 0.3026768700552094, 0.03982086875969275], [23, 798, 705.0823953200298, 334.1432979016837, 0.9478134757570267, 0.31882536783227317, 0.0416785470276751], [46, 731, 679.6849616043089, 319.8946630247399, 0.941302753763066, 0.3375635136653067, 0.04367973453080106], [38, 693, 661.2234869938007, 308.2878616086368, 0.9324770449708091, 0.3612292355312738, 0.045811380753927544], [40, 665, 645.7656636747749, 297.8057106401598, 0.9223336804421451, 0.3863943347437261, 0.0479439676739182], [24, 639, 631.3836409147973, 287.5325452094338, 0.9108013783595486, 0.4128448245749807, 0.050119050079261845], [27, 618, 623.6519062582148, 280.69613518972136, 0.9001692526648122, 0.43554025825045667, 0.051796342795844184], [26, 607, 618.1991892236267, 274.21658947806816, 0.8871463898956145, 0.4614881178266434, 0.053551558869901816], [37, 605, 613.1480790457492, 268.0665022660326, 0.874394005060664, 0.48521657423667175, 0.0551367545298495], [40, 598, 605.9192816957346, 261.3910686268521, 0.8627917167293939, 0.5055595450025112, 0.05661543415522648], [21, 593, 594.614044958479, 253.7253176933828, 0.8534117881830392, 0.5212372970061023, 0.05803048920915325], [26, 605, 585.7606828381125, 246.95182816928235, 0.8431833525348875, 0.5376261098644927, 0.059379445606567674], [29, 602, 577.5147629228793, 240.5149201513172, 0.8329308117909889, 0.5533771433382502, 0.06067155621281635], [33, 583, 571.6406726458698, 236.08020414701755, 0.8259741318066384, 0.563708021573464, 0.06154908740804832], [21, 570, 567.3196075776277, 232.4840346003638, 0.8195875181999679, 0.5729540121255963, 0.06228766590178487], [35, 579, 566.3049097540636, 230.119429704928, 0.8127050489633398, 0.5826752984205649, 0.06287010874550479], [26, 583, 564.4748875421996, 227.37174659774954, 0.805604471043015, 0.5924537418529005, 0.06349813675969958], [23, 583, 563.6774808520777, 226.10447244882855, 0.8022476686740789, 0.596991355135906, 0.06378590915934075], [31, 580, 564.080332543945, 224.7115091674152, 0.7967358413436934, 0.6043277249294096, 0.06415372113277973], [22, 585, 568.4578596050166, 223.70995761239345, 0.7870766630540902, 0.6168551908476069, 0.06456520665941605], [31, 585, 577.8133836442377, 224.06099858209475, 0.7755479707616123, 0.6312886384590999, 0.06478526797040372], [30, 587, 583.890447182401, 222.41066605719735, 0.7618232739735805, 0.647784917416402, 0.06528385662958673], [26, 588, 588.7332941603438, 219.59937836428642, 0.746006317436084, 0.6659388668229635, 0.06591950755326545], [30, 577, 593.3288656184958, 216.06757122155085, 0.7283231399716865, 0.6852338314632338, 0.06660820514100348], [35, 593, 599.1144178234287, 211.6808003997262, 0.7066456559959232, 0.7075676058597443, 0.06735737543379645], [30, 596, 604.0168762582872, 206.10509353551743, 0.6824481289737462, 0.7309340266126856, 0.06818213947025628], [32, 592, 608.7374278541419, 199.4518516759647, 0.655296824376486, 0.755371479447095, 0.06904327133528734], [27, 601, 612.3128676974197, 191.1678664906273, 0.6244123766645868, 0.7810948622722358, 0.07000374414804682], [42, 616, 615.8680691275756, 181.84482021553856, 0.5905317366855395, 0.8070144162077649, 0.07095007744957708], [34, 627, 618.3707451800456, 170.97563369825642, 0.5529874594842773, 0.8331895760588501, 0.07194548054874533], [20, 629, 618.4372143951659, 158.1667169894033, 0.5115045256262303, 0.8592805829668715, 0.07305934306492255], [30, 616, 619.7894474788376, 144.59687120559633, 0.4665999777627176, 0.8844684622708888, 0.07404149145837104], [29, 618, 621.1703968707503, 130.81236339752377, 0.421180288231741, 0.906976937306031, 0.07489430899119741], [24, 632, 624.4337280335847, 117.29549814286459, 0.3756859787578801, 0.9267470233913533, 0.07550809173882983], [41, 640, 629.4098427218046, 104.94392577019538, 0.33346769830093037, 0.9427615256202808, 0.07585605127767316], [32, 644, 634.539318869226, 92.81428693877679, 0.292540695836392, 0.9562530738667238, 0.07608748417213271], [29, 642, 639.7491371710681, 80.84433785369184, 0.2527376221597753, 0.9675348543308522, 0.07622273473259866], [27, 663, 647.2454425303501, 70.95815129944322, 0.2192619573249939, 0.9756660258869378, 0.07609781062191172], [47, 669, 656.4827710203692, 64.34068882780821, 0.19601638205311517, 0.9806006210312165, 0.07575136917505415], [35, 664, 662.0507521335019, 61.04344120355877, 0.1844071349721824, 0.9828499420416887, 0.07551861872819507], [22, 680, 670.304518327072, 62.40272077696397, 0.1861921531804888, 0.9825133495754714, 0.0750393784080454], [29, 694, 682.0072246014435, 69.20272615970028, 0.20293839614423875, 0.9791915069946248, 0.07426691820682377], [29, 708, 697.0073345999263, 79.48210766715715, 0.22806677554628293, 0.9736454929243606, 0.07325509335777446], [44, 740, 714.6992438595885, 93.59099664840414, 0.2619031640301868, 0.965094157412102, 0.07202433435919894], [40, 748, 731.6274379687737, 111.39312959956871, 0.3045077967792756, 0.9525098433615422, 0.07072058026429943], [33, 766, 749.827523794892, 133.70341236237903, 0.3566244452743036, 0.9342478284870641, 0.06918412154422196], [36, 796, 771.5340044686911, 160.51209558556488, 0.4160856025940162, 0.9093254485133326, 0.06728809113482773], [48, 819, 794.5241232990452, 189.3899360577302, 0.47673803854146063, 0.879045415554646, 0.06519408346187966], [34, 833, 818.010093419614, 218.72510326821455, 0.5347736049413643, 0.8449953795483246, 0.062994684799746], [40, 843, 847.193014683848, 248.9107531250903, 0.5876128551838398, 0.8091422201459372, 0.060572753982945066], [40, 860, 874.9118881198643, 277.37026805620883, 0.6340530328197087, 0.7732895651514574, 0.058269995885123806], [41, 890, 900.7436152744635, 304.21621361483824, 0.6754779239198742, 0.7373802101337505, 0.056079123104305253], [54, 900, 924.6538587617463, 330.8553402860805, 0.7156306917469564, 0.698478856537385, 0.053869520553072016], [49, 906, 942.2651862480523, 353.6114325966066, 0.7505560807242178, 0.6608067566905627, 0.051904706071241036], [52, 918, 958.0265479843835, 373.250793624712, 0.779207620937027, 0.6267658920790596, 0.05013257089503061], [49, 933, 972.0301994996937, 389.97838653809777, 0.8023997335449466, 0.5967869532814859, 0.0485652750210605], [55, 948, 982.6950901029222, 404.3967604346549, 0.8230360861827457, 0.5679890851424756, 0.047121235973718645], [59, 949, 988.5808782664636, 415.50569420803885, 0.8406104211456188, 0.5416402125593942, 0.045878099438460204], [49, 963, 989.122185648845, 423.3853217819455, 0.8560829550177624, 0.5168384410316788, 0.04480314530553751], [49, 976, 986.0002242460553, 429.1875476735697, 0.8705627790333371, 0.49205736226760494, 0.04378500781905943], [47, 994, 984.6273112360902, 435.80309643026294, 0.8852143170458283, 0.46518341855346407, 0.042602221432456414], [43, 1020, 985.5537594129897, 442.48941407857313, 0.8979508420568073, 0.44009576827035113, 0.04141803666443706], [41, 1039, 988.8460923441614, 448.83159660258946, 0.9077885832335909, 0.41942804883644996, 0.04036644054047013], [50, 1029, 997.0593149933917, 457.6407733385144, 0.9179810397569929, 0.3966242688574031, 0.03909175929838142], [53, 1045, 1006.5628540930174, 466.9240025534201, 0.9277592564731605, 0.3731792625915428, 0.0377393462588644], [53, 1051, 1017.1405338390672, 476.18052482620755, 0.9363121594002844, 0.3511688200270292, 0.036418626668527694], [45, 1048, 1028.7149214037931, 484.88638483868, 0.9427031235767435, 0.33363276337711756, 0.03529741475221334], [48, 1044, 1042.5732809167782, 494.2507069607704, 0.9481361473721158, 0.31786450894423957, 0.03422345063429561], [46, 1059, 1061.073689853978, 505.57832577399586, 0.9529561058922724, 0.3031083308697319, 0.03312700969969461], [62, 1075, 1085.030100296678, 519.0208038665867, 0.9566938349907006, 0.29109604272608763, 0.03210356994075261], [63, 1090, 1106.5207916475836, 529.9657261916714, 0.9578956494844797, 0.28711656987834633, 0.03157224108655273], [64, 1151, 1128.466224271798, 541.0236942298815, 0.9588655514771927, 0.28386062458597466, 0.03108596587521241], [54, 1156, 1148.5726620458995, 551.0327546761835, 0.9595087413880359, 0.2816788511762071, 0.030694032893099868], [59, 1176, 1166.961944162921, 559.4729132566381, 0.958853741641003, 0.28390051451353493, 0.03057108231426111], [51, 1186, 1185.8114122404581, 567.6802127950232, 0.9574544601868097, 0.28858440129082796, 0.030576282683405932], [60, 1182, 1203.9021559263047, 575.9448808144618, 0.9567968260200002, 0.2907573450801408, 0.030459714094894914], [50, 1187, 1219.252235759979, 582.9780478837621, 0.956287847231845, 0.2924270049716413, 0.030354146730992328], [54, 1208, 1235.3059949191388, 590.3950236775701, 0.9558684667699947, 0.2937949527047377, 0.030226715758160834], [78, 1247, 1253.5869690944037, 598.9127657409588, 0.9555184929428802, 0.2949311947627242, 0.030063475913401045], [61, 1321, 1273.595021893862, 608.9224086853463, 0.9562261130384543, 0.2926288105149093, 0.029709745797680032], [88, 1420, 1308.5422963048659, 627.5504484106456, 0.9591595933624113, 0.2828654706054685, 0.02881722497428755], [70, 1862, 1399.2722750017035, 674.6843561972529, 0.9643360598943211, 0.2646808712156895, 0.026956669529719222], [60, 1804, 1515.6689728755591, 734.8827051341807, 0.9697139920202308, 0.24424326742039024, 0.02488084878615473], [59, 1756, 1619.6537389776506, 789.1638298317597, 0.9744846208052983, 0.22445428000809697, 0.023073258302404975], [64, 1693, 1709.8520717742715, 836.9256661222696, 0.9789451145370867, 0.20412364567577593, 0.02141526535278654], [52, 1635, 1787.62550343858, 878.0258301199656, 0.9823375516080325, 0.18711743558721794, 0.020052795683018848], [56, 1584, 1856.274143641366, 914.4970016660646, 0.9853038192646898, 0.1708109590817164, 0.01880151646155571], [74, 1532, 1912.3696124229427, 944.7984278211516, 0.9880918643379891, 0.15386509555151814, 0.017580863811654805], [114, 1461, 1947.9050194228269, 964.9516764535966, 0.9907584474930058, 0.13563811677124177, 0.01635547112570971], [131, 1398, 1954.9580709104546, 970.394333926412, 0.9927520680527784, 0.12018041178549853, 0.015367534316358969], [441, 1338, 1920.0845665614802, 954.4288042713561, 0.9941528835686266, 0.10798168405884696, 0.01469842975237255]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}}, 'bounds': [0.35420247908907687, 0.5946194015953628], 'bestTools': ['DecisionTool2', 'DecisionTool1', 'DecisionTool2'], '_className': 'ConfidenceDecisionNode', '_version': 1}, 'state': {}, 'minCasesPerPartition': 70, 'curvesPerTool': {}, '_className': 'DefaultDispatchConfiguration', '_version': 1}, 'isEditable': True, '_className': 'CompositeDecisionTool', '_version': 1}, 'CompositeDecisionTool1': {'id': 'CompositeDecisionTool1', 'name': 'initial scenario', 'comments': [], 'type': 'o', 'hasHuman': False, 'tools': ['DecisionTool1', 'DecisionTool2'], 'confidenceProvider': 'DecisionTool2', 'attributes': [], 'useDeciles': False, 'dispatchConfiguration': {'tools': ['DecisionTool1', 'DecisionTool2'], 'useDeciles': False, 'decisionTree': {'bestTool': 'DecisionTool2', 'basisCount': 4961, 'basisPerf': 354.3628381374529, 'basisStdDev': 1054.9647523552628, 'basisConfidence95': 29.35684686445437, 'secondTool': 'DecisionTool1', 'secondPerf': 247.8076416045064, 'secondStdDev': 1025.617558809876, 'secondConfidence95': 28.540192976360014, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[247.8076416045064, 247.82764160451526, -0.019999999999999477, 4961, 1025.617558809876, 28.540192976360014]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[354.3628381374529, 354.4128381374724, -0.050000000000004464, 4961, 1054.9647523552628, 29.35684686445437]], '_className': 'AnalysisResult', '_version': 1}}, 'slots': [{'bestTool': 'DecisionTool2', 'basisCount': 1005, 'basisPerf': 1617.3499999999997, 'basisStdDev': 1174.3660269666914, 'basisConfidence95': 72.60667011306258, 'secondTool': 'DecisionTool1', 'secondPerf': 1346.2391542288533, 'secondStdDev': 1035.9555365441595, 'secondConfidence95': 64.04926587321648, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[1346.2391542288533, 1346.2591542288556, -0.01999999999999966, 1005, 1035.9555365441595, 64.04926587321648]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[1617.3499999999997, 1617.3999999999996, -0.04999999999999929, 1005, 1174.3660269666914, 72.60667011306258]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 0, 'lowBound': 0, 'highBound': 0.27917221855045943, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool1', 'basisCount': 860, 'basisPerf': 467.61280232558113, 'basisStdDev': 1092.9539776791103, 'basisConfidence95': 73.04806575573136, 'secondTool': 'DecisionTool2', 'secondPerf': 158.16261627906906, 'secondStdDev': 1565.5212362476395, 'secondConfidence95': 104.63230890128739, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[467.61280232558113, 467.63280232558157, -0.019999999999999678, 860, 1092.9539776791103, 73.04806575573136]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[158.16261627906906, 158.21261627906978, -0.04999999999999964, 860, 1565.5212362476395, 104.63230890128739]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 1, 'lowBound': 0.27917221855045943, 'highBound': 0.5516964465803048, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool2', 'basisCount': 3096, 'basisPerf': -1.1183979328165792, 'basisStdDev': 6.49315149684876, 'basisConfidence95': 0.22872368069351792, 'secondTool': 'DecisionTool1', 'secondPerf': -169.8138404392846, 'secondStdDev': 652.5765143911939, 'secondConfidence95': 22.98725085625042, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-169.8138404392846, -169.79384043927658, -0.02000000000000132, 3096, 652.5765143911939, 22.98725085625042]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-1.1183979328165792, -1.0683979328165374, -0.05000000000000031, 3096, 6.49315149684876, 0.22872368069351792]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 2, 'lowBound': 0.5516964465803048, 'highBound': 1, '_className': 'DecisionNode', '_version': 1}], 'confidenceProvider': 'DecisionTool2', 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 467826.4981191129, 1603.1184028439893, 982.1881055844433, 2.5426044342143435], [101, 504, 666.3114308957505, 528490.8031842152, 1586.3176847311258, 983.9435041094891, 2.381784808287103], [65, 551, 721.1424425150965, 566587.6812111209, 1571.3613505677413, 981.8584869816867, 2.2870201568194934], [53, 593, 757.4765311869324, 588438.156782675, 1553.6802331304395, 980.0049925071575, 2.229387822618676], [54, 620, 780.4182089417114, 598058.9204544935, 1532.6626508766194, 980.4135923027255, 2.196832961813439], [39, 648, 787.2394933018742, 595189.2995086145, 1512.0920750869486, 982.99257861464, 2.1901696389323777], [32, 674, 783.295106379799, 584650.681144051, 1492.7979924352276, 984.5659628102719, 2.197433664610511], [31, 705, 772.3184664651072, 568744.2484563492, 1472.8231245317365, 985.2064199678813, 2.2137138305004003], [34, 743, 755.5948864772723, 548442.8776997251, 1451.6849902377464, 985.6964857268364, 2.238634386103309], [25, 770, 733.2801280101717, 525382.706267886, 1432.965891748803, 987.0907337378834, 2.2740481234839773], [23, 798, 705.0823953200298, 498681.5183528668, 1414.5340223010965, 987.8226867193093, 2.3199340214250355], [46, 731, 679.6849616043089, 473633.6838748095, 1393.685929895692, 989.2505121642839, 2.3645874635475557], [38, 693, 661.2234869938007, 451653.2939665361, 1366.1138869096783, 995.1645783091144, 2.404525517584991], [40, 665, 645.7656636747749, 432653.36991052685, 1339.9701911943798, 1001.9207839183987, 2.441379478029498], [24, 639, 631.3836409147973, 415500.70557218353, 1316.1592371008348, 1007.9690372259919, 2.476469619663136], [27, 618, 623.6519062582148, 402465.5586253533, 1290.6737062348298, 1013.8865998423411, 2.499076990401595], [26, 607, 618.1991892236267, 389261.09536828665, 1259.3387443847837, 1024.7349827706264, 2.5234670426113603], [37, 605, 613.1480790457492, 375796.3208274763, 1225.7930300045407, 1033.1961436571785, 2.544279235912517], [40, 598, 605.9192816957346, 361413.5670403987, 1192.9429478756358, 1043.4801953459466, 2.5721174174775183], [21, 593, 594.614044958479, 344399.7735477707, 1158.397708455809, 1047.862698922668, 2.6019004740857268], [26, 605, 585.7606828381125, 327674.96602411446, 1118.8015024718702, 1048.905316032369, 2.6227935423084596], [29, 602, 577.5147629228793, 311993.0060445265, 1080.4676384911384, 1047.583031697504, 2.639786197418429], [33, 583, 571.6406726458698, 300351.63411547506, 1050.8406713793868, 1040.2590377434285, 2.644023186756073], [21, 570, 567.3196075776277, 289996.6168048502, 1022.3394817714607, 1028.5211109447323, 2.639057055224421], [35, 579, 566.3049097540636, 280054.13229566737, 989.0577583630346, 1015.7990027303528, 2.6250331907656013], [26, 583, 564.4748875421996, 269415.4618561862, 954.5702308538748, 1001.1200460164004, 2.6102183429752492], [23, 583, 563.6774808520777, 262037.12118649483, 929.7413151591187, 980.5795145693014, 2.5851285079122257], [31, 580, 564.080332543945, 256038.50452377542, 907.8086568594504, 964.0925117557081, 2.562388327619304], [22, 585, 568.4578596050166, 252014.84753109206, 886.6614939802236, 949.0063409865721, 2.532453564111772], [31, 585, 577.8133836442377, 248931.0482763152, 861.631299387081, 937.4368966797739, 2.49650990995888], [30, 587, 583.890447182401, 244790.8083623263, 838.4819773763359, 930.9642464701626, 2.474895622580027], [26, 588, 588.7332941603438, 240378.78079377607, 816.596524022329, 928.9921367769045, 2.4620835810618735], [30, 577, 593.3288656184958, 235536.43598308324, 793.949020961777, 932.9649826612184, 2.457768673297603], [35, 593, 599.1144178234287, 230494.56677397917, 769.4509092649166, 943.4407064593674, 2.459566058097794], [30, 596, 604.0168762582872, 226159.54573746584, 748.8517444693265, 954.2564264250507, 2.463565318405578], [32, 592, 608.7374278541419, 221760.42607211822, 728.5913956493364, 968.5041426544662, 2.4722467250018108], [27, 601, 612.3128676974197, 214981.59918010086, 702.1952682083306, 985.9859259686688, 2.4871657937543765], [42, 616, 615.8680691275756, 206908.0703074762, 671.9233572234955, 1002.0943754879978, 2.500152729613687], [34, 627, 618.3707451800456, 197230.31106423997, 637.9031110432437, 1010.8706595904476, 2.505990379924332], [20, 629, 618.4372143951659, 188064.72417361918, 608.1934262560419, 1021.3655439800741, 2.51883004540106], [30, 616, 619.7894474788376, 177899.49697519335, 574.0642977993512, 1035.7689439546082, 2.533759704623051], [29, 618, 621.1703968707503, 166196.271787549, 535.1068647984208, 1050.636949761916, 2.549042235188664], [24, 632, 624.4337280335847, 153294.2220726125, 490.98636153224476, 1068.9840724434248, 2.5644752505411077], [41, 640, 629.4098427218046, 140536.38032981675, 446.5655628201957, 1086.1740894908082, 2.574773469871046], [32, 644, 634.539318869226, 126730.03639650389, 399.439507144622, 1103.6411155715418, 2.584882093820425], [29, 642, 639.7491371710681, 111900.09151730503, 349.8249079697722, 1119.231279228897, 2.5924544614604237], [27, 663, 647.2454425303501, 96008.23885518893, 296.667176148365, 1132.6382819920025, 2.5927890800267797], [47, 669, 656.4827710203692, 80018.982484126, 243.7809064196848, 1140.681042408093, 2.583607382716839], [35, 664, 662.0507521335019, 63307.23640573951, 191.2458711110222, 1149.6858627916233, 2.582854996589537], [22, 680, 670.304518327072, 45210.38674241446, 134.8950678573653, 1152.4755598024128, 2.5700161948760254], [29, 694, 682.0072246014435, 29427.989708740246, 86.2981758761796, 1144.9890768167488, 2.5395820893522374], [29, 708, 697.0073345999263, 15095.865671981548, 43.31623190348891, 1130.9627478680466, 2.4966723256078565], [44, 740, 714.6992438595885, 2347.880837814503, 6.570262548859708, 1107.5136519951589, 2.4398827368246563], [40, 748, 731.6274379687737, -10839.152799762986, -29.630252331311848, 1081.6125253859916, 2.383125606504419], [33, 766, 749.827523794892, -23663.01367264134, -63.11588444468497, 1055.4735520155316, 2.325407452733299], [36, 796, 771.5340044686911, -35436.17430297201, -91.85900841110629, 1026.1533053738342, 2.260396674197986], [48, 819, 794.5241232990452, -45646.33711114669, -114.90233152799112, 996.3497232826718, 2.1948681160633265], [34, 833, 818.010093419614, -56185.87099055746, -137.37207264931837, 971.3039878519611, 2.1357692333699165], [40, 843, 847.193014683848, -66247.11567305274, -156.39202525241447, 950.8268097504587, 2.076421884959117], [40, 860, 874.9118881198643, -75591.96772642512, -172.7990412585842, 936.4091621882596, 2.0277141868069837], [41, 890, 900.7436152744635, -87351.69600789635, -193.95462710280685, 928.5841500444686, 1.990059715247872], [54, 900, 924.6538587617463, -100286.37988653741, -216.9165876208775, 926.8120592273824, 1.9622860486844365], [49, 906, 942.2651862480523, -113207.61127880005, -240.2882180749444, 931.6714037305406, 1.9489508252305945], [52, 918, 958.0265479843835, -124215.54968756462, -259.31546458478607, 934.9244762825593, 1.9362238448406883], [49, 933, 972.0301994996937, -135062.36575006542, -277.8974682465264, 938.48780435743, 1.9258857024456595], [55, 948, 982.6950901029222, -145348.1116388483, -295.8152800450551, 943.8754383887951, 1.9208967456341817], [59, 949, 988.5808782664636, -154256.8502912457, -312.07734983048516, 946.8174962197812, 1.9181523833853813], [49, 963, 989.122185648845, -160653.81202853814, -324.8411861738848, 948.6834844433675, 1.9195161474435625], [49, 976, 986.0002242460553, -165108.73034262026, -334.9060705718816, 949.2474436157664, 1.923123977200898], [47, 994, 984.6273112360902, -168196.46206225042, -341.644925227797, 944.2462219394298, 1.9193879339183886], [43, 1020, 985.5537594129897, -167578.5446045452, -340.0698196399909, 932.576441850336, 1.9065936054309776], [41, 1039, 988.8460923441614, -164635.92830396976, -332.9859511578457, 914.5315477256437, 1.884911943450251], [50, 1029, 997.0593149933917, -161375.17509093462, -323.7022565543238, 892.2953106178265, 1.854171404694584], [53, 1045, 1006.5628540930174, -158723.01371965124, -315.3762590666464, 871.7975409154251, 1.824078151841528], [53, 1051, 1017.1405338390672, -154576.75817184622, -303.9437580732644, 849.8834835448015, 1.7916174560804736], [45, 1048, 1028.7149214037931, -150051.40252755745, -291.72591824136475, 828.3907551395513, 1.7588393502984483], [48, 1044, 1042.5732809167782, -144959.3098066014, -278.07984812181763, 807.0859922294462, 1.7244979440054986], [46, 1059, 1061.073689853978, -139812.68745392922, -263.5305894224365, 782.636062113168, 1.683306599207149], [62, 1075, 1085.030100296678, -135614.4811183681, -249.97367553450775, 755.7472138909835, 1.6357745824443095], [63, 1090, 1106.5207916475836, -131106.66390561414, -236.97099032436506, 731.8360066393614, 1.5939811033911373], [64, 1151, 1128.466224271798, -128588.81348484031, -227.90015459757157, 715.5016526421232, 1.5606916596909137], [54, 1156, 1148.5726620458995, -125814.42937593175, -219.07961687304015, 700.4481253919654, 1.5306109729163493], [59, 1176, 1166.961944162921, -122472.89823366192, -209.90041508425287, 685.2829227990027, 1.5019749145507262], [51, 1186, 1185.8114122404581, -118717.30206073177, -200.22965006961553, 669.7786351977905, 1.4730378409551832], [60, 1182, 1203.9021559263047, -114401.96790785609, -190.05193627190258, 650.7489088735505, 1.4410106799227245], [50, 1187, 1219.252235759979, -108373.65953890502, -177.7706964323982, 625.0622306098048, 1.4033659007851016], [54, 1208, 1235.3059949191388, -103776.44535246746, -168.01739128491886, 601.6735985325691, 1.3678840147182045], [78, 1247, 1253.5869690944037, -100427.21257941338, -160.22376596968363, 581.2044137220811, 1.3345759327951405], [61, 1321, 1273.595021893862, -95876.92939569919, -150.56109320076993, 558.174087267923, 1.2975533711981428], [88, 1420, 1308.5422963048659, -91440.31617761923, -139.7590531629485, 530.6416030589909, 1.2481386649201143], [70, 1862, 1399.2722750017035, -86630.53867037191, -123.82227564719875, 482.8358147514897, 1.1513429212724815], [60, 1804, 1515.6689728755591, -83167.88371089716, -109.74412645408886, 435.5787665251204, 1.0507205870097283], [59, 1756, 1619.6537389776506, -79313.1373580395, -97.93838701363802, 392.35119564423746, 0.9646782183078773], [64, 1693, 1709.8520717742715, -75042.7754339379, -87.77692137550572, 350.55529647920315, 0.8874728350717506], [52, 1635, 1787.62550343858, -71425.04872290586, -79.91052777611026, 314.9770178586767, 0.8227298107731542], [56, 1584, 1856.274143641366, -69018.3592692434, -74.36224816863884, 288.01044072418273, 0.7720388044941164], [74, 1532, 1912.3696124229427, -65701.19108220657, -68.71181246073469, 260.6564084803957, 0.7236096630146555], [114, 1461, 1947.9050194228269, -61393.1758181778, -63.035081491159026, 231.16393640836571, 0.6751996735769568], [131, 1398, 1954.9580709104546, -57303.86226840527, -58.6241343188685, 204.51566071354222, 0.6339436156717544], [441, 1338, 1920.0845665614802, -53003.56544505126, -55.20961562643143, 180.53710663190884, 0.6010064487032096]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 579090.1018226708, 1984.3895180566637, 794.1976618676379, 2.2863660720641907], [101, 504, 666.3114308957505, 654994.6039346324, 1966.0314188339694, 801.8915402948611, 2.1501821655920383], [65, 551, 721.1424425150965, 700409.8245314332, 1942.5006302184765, 810.4019333318075, 2.077761895110684], [53, 593, 757.4765311869324, 726574.1445434467, 1918.4070123068684, 815.8796867419061, 2.0341573335795933], [54, 620, 780.4182089417114, 739018.0882801425, 1893.9027301330918, 822.1314143978245, 2.011698981557118], [39, 648, 787.2394933018742, 735612.8196445594, 1868.8412507335477, 840.7289231948547, 2.025492451585069], [32, 674, 783.295106379799, 721929.4287311737, 1843.3140277557902, 863.7344179879951, 2.0581805868354692], [31, 705, 772.3184664651072, 702172.1661282271, 1818.34876833144, 882.8109481073785, 2.095519550380126], [34, 743, 755.5948864772723, 676274.2701226803, 1790.0445919521774, 907.4188431349253, 2.1479069486948172], [25, 770, 733.2801280101717, 644739.3560722482, 1758.5076465165157, 942.2270607916229, 2.221768985590072], [23, 798, 705.0823953200298, 608241.8258541685, 1725.3070843673345, 978.678444974414, 2.309171279904825], [46, 731, 679.6849616043089, 574093.7656300052, 1689.2937112362488, 1020.7037521045437, 2.4018843723335563], [38, 693, 661.2234869938007, 545357.8380806628, 1649.5416415411623, 1073.1591706385566, 2.4969739191314835], [40, 665, 645.7656636747749, 518561.99321640324, 1606.0376770901373, 1128.6049021662166, 2.5911321497962345], [24, 639, 631.3836409147973, 492622.8116804608, 1560.4547845639804, 1185.6431299460257, 2.6858787510025888], [27, 618, 623.6519062582148, 472442.43322562525, 1515.0837461884282, 1233.3592722467988, 2.756320694820169], [26, 607, 618.1991892236267, 451891.35768006, 1461.960369917578, 1288.3657278045728, 2.8295113316257474], [37, 605, 613.1480790457492, 431580.52109004033, 1407.7529909633413, 1337.7035520108868, 2.895031713116739], [40, 598, 605.9192816957346, 410948.6485813888, 1356.4468436498732, 1379.1350773602337, 2.9570051927779004], [21, 593, 594.614044958479, 388620.7694368255, 1307.1361927347757, 1409.6058752106946, 3.0177783629418573], [26, 605, 585.7606828381125, 366286.03881393553, 1250.6337470081328, 1437.2863518038662, 3.0702066869998097], [29, 602, 577.5147629228793, 344131.8366232376, 1191.768102624911, 1461.8529369654677, 3.1183608429800618], [33, 583, 571.6406726458698, 326225.1250204127, 1141.3642892499656, 1477.429085344917, 3.1509958617514235], [21, 570, 567.3196075776277, 310508.7400128223, 1094.6518888661349, 1487.1821593363147, 3.1733959115970896], [35, 579, 566.3049097540636, 296062.96903519187, 1045.5956285590632, 1494.8083790186467, 3.184371070011158], [26, 583, 564.4748875421996, 281138.07672059524, 996.1048150244865, 1498.7881402802714, 3.1937718041559697], [23, 583, 563.6774808520777, 270394.33250636276, 959.3937728277658, 1487.0189109094508, 3.1834569019613417], [31, 580, 564.080332543945, 259014.20464295542, 918.3592821782233, 1480.1177610754355, 3.174926878589568], [22, 585, 568.4578596050166, 247826.1144449741, 871.9243133243754, 1479.2362835386384, 3.161736763371161], [31, 585, 577.8133836442377, 237770.845197093, 823.0022077283332, 1482.5594840883643, 3.139556741809795], [30, 587, 583.890447182401, 226234.50715345488, 774.9210772163282, 1493.569002256613, 3.1347508743112242], [26, 588, 588.7332941603438, 214488.64727069048, 728.6445302081851, 1508.9178795228713, 3.1378311877652023], [30, 577, 593.3288656184958, 202999.86215278617, 684.2743507554643, 1522.9398467301141, 3.1401450359625707], [35, 593, 599.1144178234287, 190070.1051476771, 634.5035255142021, 1546.1946332480097, 3.148714372821247], [30, 596, 604.0168762582872, 176600.79963039325, 584.7545211795571, 1571.1128884385575, 3.161078133010358], [32, 592, 608.7374278541419, 161771.98393341136, 531.5000410067546, 1598.4175434141368, 3.176041639809634], [27, 601, 612.3128676974197, 144058.39661803457, 470.5385243977019, 1631.566384863241, 3.199423664600885], [42, 616, 615.8680691275756, 125165.46440896092, 406.4684327156867, 1665.8977676576399, 3.2235647684050734], [34, 627, 618.3707451800456, 103216.15749851275, 333.83260221490656, 1698.9680532185037, 3.248809228290445], [20, 629, 618.4372143951659, 79967.66718332618, 258.61208000406276, 1733.0907058814907, 3.2810957955653532], [30, 616, 619.7894474788376, 55025.287664437405, 177.56122789204545, 1764.1801822072935, 3.306781140761259], [29, 618, 621.1703968707503, 29917.36028218845, 96.32577609268617, 1781.4209543585068, 3.319204223367515], [24, 632, 624.4337280335847, 5200.38395836372, 16.656319877980778, 1786.8016845313803, 3.3155155682668016], [41, 640, 629.4098427218046, -15480.94780890317, -49.19194698944566, 1773.6209707200735, 3.290180422310677], [32, 644, 634.539318869226, -34126.932447449304, -107.56443748281774, 1746.5810604088797, 3.2517801492008664], [29, 642, 639.7491371710681, -50990.26837358493, -159.40707196280343, 1709.4617450597777, 3.2039144426579127], [27, 663, 647.2454425303501, -65692.06383754365, -202.98965283008005, 1656.6005024636322, 3.1356708245488156], [47, 669, 656.4827710203692, -76769.6429633882, -233.88166865084784, 1586.263367604898, 3.0467165552414204], [35, 664, 662.0507521335019, -80916.02683970236, -244.44055558866197, 1501.6100462225154, 2.951814163798141], [22, 680, 670.304518327072, -80244.3187812565, -239.42645942929974, 1393.7986280715616, 2.8263114261957165], [29, 694, 682.0072246014435, -73224.96716086917, -214.7336993494781, 1263.2980518182828, 2.667561845037861], [29, 708, 697.0073345999263, -64395.24704142512, -184.77638281493037, 1129.636697362568, 2.495208225828501], [44, 740, 714.6992438595885, -54871.66335039772, -153.55175990973328, 993.3453418647225, 2.3107051901402733], [40, 748, 731.6274379687737, -46233.842898913594, -126.38630127725439, 858.8932127643794, 2.1236388639957853], [33, 766, 749.827523794892, -37523.07749585744, -100.08455626155839, 726.2167107950241, 1.9288946185490463], [36, 796, 771.5340044686911, -28400.66001668654, -73.62127878276566, 589.8699596046911, 1.7137863329435834], [48, 819, 794.5241232990452, -19391.464168971605, -48.812776353356846, 451.3258881080688, 1.477228405874995], [34, 833, 818.010093419614, -11353.337525763001, -27.758428941387375, 308.1377855278013, 1.2029545487390985], [40, 843, 847.193014683848, -6004.944398737014, -14.17609516286655, 174.01586830068567, 0.8882992893651278], [40, 860, 874.9118881198643, -2846.5645901243543, -6.507088608068771, 49.62334849556987, 0.4667850580636025], [41, 890, 900.7436152744635, -2250.711662587046, -4.997452381388763, 13.491322241149065, 0.23987375460016266], [54, 900, 924.6538587617463, -2025.541110824947, -4.381187817758005, 12.783821516732514, 0.23046067386731578], [49, 906, 942.2651862480523, -1824.6502718914305, -3.872901808368604, 12.19677126479848, 0.2229933634841931], [52, 918, 958.0265479843835, -1653.4773958526744, -3.4518404512515186, 11.683744094676932, 0.2164503624768715], [49, 933, 972.0301994996937, -1502.539139592803, -3.091548267463631, 11.194740084381769, 0.2103406442206863], [55, 948, 982.6950901029222, -1362.7777925119017, -2.7735516463589365, 10.72028088224095, 0.20471504468184287], [59, 949, 988.5808782664636, -1223.5014614341126, -2.475268313058202, 10.150280409644534, 0.19860445608271252], [49, 963, 989.122185648845, -1082.7106102596933, -2.1892353158562634, 9.52054284334566, 0.19229232730283854], [49, 976, 986.0002242460553, -947.7944509449587, -1.9225035200568832, 8.876600727164178, 0.1859691505258136], [47, 994, 984.6273112360902, -825.8896024093177, -1.6775679345568935, 8.23032214670349, 0.17919609840337328], [43, 1020, 985.5537594129897, -730.3436699132553, -1.4820980853408925, 7.662971190544345, 0.17282816080077723], [41, 1039, 988.8460923441614, -657.5192629299321, -1.3298717930334638, 7.175897363106353, 0.16696668694222805], [50, 1029, 997.0593149933917, -580.8032145889193, -1.1650324225550588, 6.601893064880426, 0.15948866268443604], [53, 1045, 1006.5628540930174, -511.63770149355565, -1.0166035820079542, 6.042425757070596, 0.151859261377455], [53, 1051, 1017.1405338390672, -454.6832687742445, -0.8940421773539994, 5.546049316486426, 0.14472963074830972], [45, 1048, 1028.7149214037931, -407.4847142955956, -0.7922208686144816, 5.084077822417897, 0.13778903119014485], [48, 1044, 1042.5732809167782, -374.2743477445835, -0.7179818523940464, 4.788085878537738, 0.13282619801150347], [46, 1059, 1061.073689853978, -350.04926072988974, -0.6598019799700481, 4.570279178811609, 0.1286336708537136], [62, 1075, 1085.030100296678, -328.6461850027627, -0.6057826136121044, 4.329083504840714, 0.12380356759976449], [63, 1090, 1106.5207916475836, -317.19560325890706, -0.5733206382622241, 4.169318812809933, 0.12031196905800855], [64, 1151, 1128.466224271798, -306.7713945414167, -0.5436961921290592, 4.04067367906307, 0.1172839754156031], [54, 1156, 1148.5726620458995, -298.00577696977024, -0.5189149747634534, 3.9341680614154635, 0.11471053042699211], [59, 1176, 1166.961944162921, -299.31283109362795, -0.5129778783117549, 3.91398747996473, 0.11351086695396706], [51, 1186, 1185.8114122404581, -304.4826265474695, -0.5135430868761561, 3.911059037327626, 0.11256294429204221], [60, 1182, 1203.9021559263047, -307.82874621129844, -0.51138499037648, 3.912714087566298, 0.1117376497357607], [50, 1187, 1219.252235759979, -312.59110762263106, -0.5127587195733755, 3.940319795167894, 0.11142304588895659], [54, 1208, 1235.3059949191388, -314.4260738225938, -0.5090658915537372, 3.920692806121318, 0.11042062595295946], [78, 1247, 1253.5869690944037, -316.04818573025295, -0.5042301707372843, 3.88785365120444, 0.10915252614201387], [61, 1321, 1273.595021893862, -316.32163218018144, -0.4967381730336927, 3.87517153252044, 0.10811497705185102], [88, 1420, 1308.5422963048659, -304.9701012906381, -0.4661218856307963, 3.741118141014307, 0.10480039037492729], [70, 1862, 1399.2722750017035, -286.0307434276999, -0.40882785793401316, 3.452370266800767, 0.09735620591583363], [60, 1804, 1515.6689728755591, -266.20066087454626, -0.3512649076262408, 3.1407287004442233, 0.08922141421573956], [59, 1756, 1619.6537389776506, -244.7605327494802, -0.30223809800726503, 2.851569139512236, 0.08224072597942804], [64, 1693, 1709.8520717742715, -220.603035778466, -0.2580375687699716, 2.5691933144059904, 0.07597578456037091], [52, 1635, 1787.62550343858, -199.92601566869484, -0.22367773930739737, 2.3083033859706235, 0.07043106324929338], [56, 1584, 1856.274143641366, -178.54679382067667, -0.192371147798708, 2.035700023968529, 0.0649070550053875], [74, 1532, 1912.3696124229427, -157.34880469258368, -0.16455898867084087, 1.7862237999055746, 0.05990155412094109], [114, 1461, 1947.9050194228269, -134.75242823321406, -0.13835626161396905, 1.5333825982636828, 0.054991738316519066], [131, 1398, 1954.9580709104546, -114.6680609492478, -0.1173099951917077, 1.2772216523868973, 0.05009801101711326], [441, 1338, 1920.0845665614802, -98.906396787632, -0.10302295899888957, 1.0918463199804667, 0.046738702326131774]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}}, 'bounds': [0.27917221855045943, 0.5516964465803048], 'bestTools': ['DecisionTool2', 'DecisionTool1', 'DecisionTool2'], '_className': 'ConfidenceDecisionNode', '_version': 1}, 'state': {}, 'minCasesPerPartition': 70, 'curvesPerTool': {}, '_className': 'DefaultDispatchConfiguration', '_version': 1}, 'isEditable': True, '_className': 'CompositeDecisionTool', '_version': 1}, 'CompositeDecisionTool2': {'id': 'CompositeDecisionTool2', 'name': 'initial scenario', 'comments': [], 'type': 'o', 'hasHuman': False, 'tools': ['DecisionTool1', 'DecisionTool2'], 'confidenceProvider': 'DecisionTool2', 'attributes': [], 'useDeciles': False, 'dispatchConfiguration': {'tools': ['DecisionTool1', 'DecisionTool2'], 'useDeciles': False, 'decisionTree': {'bestTool': 'DecisionTool2', 'basisCount': 4961, 'basisPerf': 354.3628381374529, 'basisStdDev': 1054.9647523552628, 'basisConfidence95': 29.35684686445437, 'secondTool': 'DecisionTool1', 'secondPerf': 247.8076416045064, 'secondStdDev': 1025.617558809876, 'secondConfidence95': 28.540192976360014, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[247.8076416045064, 247.82764160451526, -0.019999999999999477, 4961, 1025.617558809876, 28.540192976360014]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[354.3628381374529, 354.4128381374724, -0.050000000000004464, 4961, 1054.9647523552628, 29.35684686445437]], '_className': 'AnalysisResult', '_version': 1}}, 'slots': [{'bestTool': 'DecisionTool2', 'basisCount': 1005, 'basisPerf': 1617.3499999999997, 'basisStdDev': 1174.3660269666914, 'basisConfidence95': 72.60667011306258, 'secondTool': 'DecisionTool1', 'secondPerf': 1346.2391542288533, 'secondStdDev': 1035.9555365441595, 'secondConfidence95': 64.04926587321648, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[1346.2391542288533, 1346.2591542288556, -0.01999999999999966, 1005, 1035.9555365441595, 64.04926587321648]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[1617.3499999999997, 1617.3999999999996, -0.04999999999999929, 1005, 1174.3660269666914, 72.60667011306258]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 0, 'lowBound': 0, 'highBound': 0.27917221855045943, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool1', 'basisCount': 860, 'basisPerf': 467.61280232558113, 'basisStdDev': 1092.9539776791103, 'basisConfidence95': 73.04806575573136, 'secondTool': 'DecisionTool2', 'secondPerf': 158.16261627906906, 'secondStdDev': 1565.5212362476395, 'secondConfidence95': 104.63230890128739, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[467.61280232558113, 467.63280232558157, -0.019999999999999678, 860, 1092.9539776791103, 73.04806575573136]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[158.16261627906906, 158.21261627906978, -0.04999999999999964, 860, 1565.5212362476395, 104.63230890128739]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 1, 'lowBound': 0.27917221855045943, 'highBound': 0.5516964465803048, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool2', 'basisCount': 3096, 'basisPerf': -1.1183979328165792, 'basisStdDev': 6.49315149684876, 'basisConfidence95': 0.22872368069351792, 'secondTool': 'DecisionTool1', 'secondPerf': -169.8138404392846, 'secondStdDev': 652.5765143911939, 'secondConfidence95': 22.98725085625042, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-169.8138404392846, -169.79384043927658, -0.02000000000000132, 3096, 652.5765143911939, 22.98725085625042]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-1.1183979328165792, -1.0683979328165374, -0.05000000000000031, 3096, 6.49315149684876, 0.22872368069351792]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 2, 'lowBound': 0.5516964465803048, 'highBound': 1, '_className': 'DecisionNode', '_version': 1}], 'confidenceProvider': 'DecisionTool2', 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 467826.4981191129, 1603.1184028439893, 982.1881055844433, 2.5426044342143435], [101, 504, 666.3114308957505, 528490.8031842152, 1586.3176847311258, 983.9435041094891, 2.381784808287103], [65, 551, 721.1424425150965, 566587.6812111209, 1571.3613505677413, 981.8584869816867, 2.2870201568194934], [53, 593, 757.4765311869324, 588438.156782675, 1553.6802331304395, 980.0049925071575, 2.229387822618676], [54, 620, 780.4182089417114, 598058.9204544935, 1532.6626508766194, 980.4135923027255, 2.196832961813439], [39, 648, 787.2394933018742, 595189.2995086145, 1512.0920750869486, 982.99257861464, 2.1901696389323777], [32, 674, 783.295106379799, 584650.681144051, 1492.7979924352276, 984.5659628102719, 2.197433664610511], [31, 705, 772.3184664651072, 568744.2484563492, 1472.8231245317365, 985.2064199678813, 2.2137138305004003], [34, 743, 755.5948864772723, 548442.8776997251, 1451.6849902377464, 985.6964857268364, 2.238634386103309], [25, 770, 733.2801280101717, 525382.706267886, 1432.965891748803, 987.0907337378834, 2.2740481234839773], [23, 798, 705.0823953200298, 498681.5183528668, 1414.5340223010965, 987.8226867193093, 2.3199340214250355], [46, 731, 679.6849616043089, 473633.6838748095, 1393.685929895692, 989.2505121642839, 2.3645874635475557], [38, 693, 661.2234869938007, 451653.2939665361, 1366.1138869096783, 995.1645783091144, 2.404525517584991], [40, 665, 645.7656636747749, 432653.36991052685, 1339.9701911943798, 1001.9207839183987, 2.441379478029498], [24, 639, 631.3836409147973, 415500.70557218353, 1316.1592371008348, 1007.9690372259919, 2.476469619663136], [27, 618, 623.6519062582148, 402465.5586253533, 1290.6737062348298, 1013.8865998423411, 2.499076990401595], [26, 607, 618.1991892236267, 389261.09536828665, 1259.3387443847837, 1024.7349827706264, 2.5234670426113603], [37, 605, 613.1480790457492, 375796.3208274763, 1225.7930300045407, 1033.1961436571785, 2.544279235912517], [40, 598, 605.9192816957346, 361413.5670403987, 1192.9429478756358, 1043.4801953459466, 2.5721174174775183], [21, 593, 594.614044958479, 344399.7735477707, 1158.397708455809, 1047.862698922668, 2.6019004740857268], [26, 605, 585.7606828381125, 327674.96602411446, 1118.8015024718702, 1048.905316032369, 2.6227935423084596], [29, 602, 577.5147629228793, 311993.0060445265, 1080.4676384911384, 1047.583031697504, 2.639786197418429], [33, 583, 571.6406726458698, 300351.63411547506, 1050.8406713793868, 1040.2590377434285, 2.644023186756073], [21, 570, 567.3196075776277, 289996.6168048502, 1022.3394817714607, 1028.5211109447323, 2.639057055224421], [35, 579, 566.3049097540636, 280054.13229566737, 989.0577583630346, 1015.7990027303528, 2.6250331907656013], [26, 583, 564.4748875421996, 269415.4618561862, 954.5702308538748, 1001.1200460164004, 2.6102183429752492], [23, 583, 563.6774808520777, 262037.12118649483, 929.7413151591187, 980.5795145693014, 2.5851285079122257], [31, 580, 564.080332543945, 256038.50452377542, 907.8086568594504, 964.0925117557081, 2.562388327619304], [22, 585, 568.4578596050166, 252014.84753109206, 886.6614939802236, 949.0063409865721, 2.532453564111772], [31, 585, 577.8133836442377, 248931.0482763152, 861.631299387081, 937.4368966797739, 2.49650990995888], [30, 587, 583.890447182401, 244790.8083623263, 838.4819773763359, 930.9642464701626, 2.474895622580027], [26, 588, 588.7332941603438, 240378.78079377607, 816.596524022329, 928.9921367769045, 2.4620835810618735], [30, 577, 593.3288656184958, 235536.43598308324, 793.949020961777, 932.9649826612184, 2.457768673297603], [35, 593, 599.1144178234287, 230494.56677397917, 769.4509092649166, 943.4407064593674, 2.459566058097794], [30, 596, 604.0168762582872, 226159.54573746584, 748.8517444693265, 954.2564264250507, 2.463565318405578], [32, 592, 608.7374278541419, 221760.42607211822, 728.5913956493364, 968.5041426544662, 2.4722467250018108], [27, 601, 612.3128676974197, 214981.59918010086, 702.1952682083306, 985.9859259686688, 2.4871657937543765], [42, 616, 615.8680691275756, 206908.0703074762, 671.9233572234955, 1002.0943754879978, 2.500152729613687], [34, 627, 618.3707451800456, 197230.31106423997, 637.9031110432437, 1010.8706595904476, 2.505990379924332], [20, 629, 618.4372143951659, 188064.72417361918, 608.1934262560419, 1021.3655439800741, 2.51883004540106], [30, 616, 619.7894474788376, 177899.49697519335, 574.0642977993512, 1035.7689439546082, 2.533759704623051], [29, 618, 621.1703968707503, 166196.271787549, 535.1068647984208, 1050.636949761916, 2.549042235188664], [24, 632, 624.4337280335847, 153294.2220726125, 490.98636153224476, 1068.9840724434248, 2.5644752505411077], [41, 640, 629.4098427218046, 140536.38032981675, 446.5655628201957, 1086.1740894908082, 2.574773469871046], [32, 644, 634.539318869226, 126730.03639650389, 399.439507144622, 1103.6411155715418, 2.584882093820425], [29, 642, 639.7491371710681, 111900.09151730503, 349.8249079697722, 1119.231279228897, 2.5924544614604237], [27, 663, 647.2454425303501, 96008.23885518893, 296.667176148365, 1132.6382819920025, 2.5927890800267797], [47, 669, 656.4827710203692, 80018.982484126, 243.7809064196848, 1140.681042408093, 2.583607382716839], [35, 664, 662.0507521335019, 63307.23640573951, 191.2458711110222, 1149.6858627916233, 2.582854996589537], [22, 680, 670.304518327072, 45210.38674241446, 134.8950678573653, 1152.4755598024128, 2.5700161948760254], [29, 694, 682.0072246014435, 29427.989708740246, 86.2981758761796, 1144.9890768167488, 2.5395820893522374], [29, 708, 697.0073345999263, 15095.865671981548, 43.31623190348891, 1130.9627478680466, 2.4966723256078565], [44, 740, 714.6992438595885, 2347.880837814503, 6.570262548859708, 1107.5136519951589, 2.4398827368246563], [40, 748, 731.6274379687737, -10839.152799762986, -29.630252331311848, 1081.6125253859916, 2.383125606504419], [33, 766, 749.827523794892, -23663.01367264134, -63.11588444468497, 1055.4735520155316, 2.325407452733299], [36, 796, 771.5340044686911, -35436.17430297201, -91.85900841110629, 1026.1533053738342, 2.260396674197986], [48, 819, 794.5241232990452, -45646.33711114669, -114.90233152799112, 996.3497232826718, 2.1948681160633265], [34, 833, 818.010093419614, -56185.87099055746, -137.37207264931837, 971.3039878519611, 2.1357692333699165], [40, 843, 847.193014683848, -66247.11567305274, -156.39202525241447, 950.8268097504587, 2.076421884959117], [40, 860, 874.9118881198643, -75591.96772642512, -172.7990412585842, 936.4091621882596, 2.0277141868069837], [41, 890, 900.7436152744635, -87351.69600789635, -193.95462710280685, 928.5841500444686, 1.990059715247872], [54, 900, 924.6538587617463, -100286.37988653741, -216.9165876208775, 926.8120592273824, 1.9622860486844365], [49, 906, 942.2651862480523, -113207.61127880005, -240.2882180749444, 931.6714037305406, 1.9489508252305945], [52, 918, 958.0265479843835, -124215.54968756462, -259.31546458478607, 934.9244762825593, 1.9362238448406883], [49, 933, 972.0301994996937, -135062.36575006542, -277.8974682465264, 938.48780435743, 1.9258857024456595], [55, 948, 982.6950901029222, -145348.1116388483, -295.8152800450551, 943.8754383887951, 1.9208967456341817], [59, 949, 988.5808782664636, -154256.8502912457, -312.07734983048516, 946.8174962197812, 1.9181523833853813], [49, 963, 989.122185648845, -160653.81202853814, -324.8411861738848, 948.6834844433675, 1.9195161474435625], [49, 976, 986.0002242460553, -165108.73034262026, -334.9060705718816, 949.2474436157664, 1.923123977200898], [47, 994, 984.6273112360902, -168196.46206225042, -341.644925227797, 944.2462219394298, 1.9193879339183886], [43, 1020, 985.5537594129897, -167578.5446045452, -340.0698196399909, 932.576441850336, 1.9065936054309776], [41, 1039, 988.8460923441614, -164635.92830396976, -332.9859511578457, 914.5315477256437, 1.884911943450251], [50, 1029, 997.0593149933917, -161375.17509093462, -323.7022565543238, 892.2953106178265, 1.854171404694584], [53, 1045, 1006.5628540930174, -158723.01371965124, -315.3762590666464, 871.7975409154251, 1.824078151841528], [53, 1051, 1017.1405338390672, -154576.75817184622, -303.9437580732644, 849.8834835448015, 1.7916174560804736], [45, 1048, 1028.7149214037931, -150051.40252755745, -291.72591824136475, 828.3907551395513, 1.7588393502984483], [48, 1044, 1042.5732809167782, -144959.3098066014, -278.07984812181763, 807.0859922294462, 1.7244979440054986], [46, 1059, 1061.073689853978, -139812.68745392922, -263.5305894224365, 782.636062113168, 1.683306599207149], [62, 1075, 1085.030100296678, -135614.4811183681, -249.97367553450775, 755.7472138909835, 1.6357745824443095], [63, 1090, 1106.5207916475836, -131106.66390561414, -236.97099032436506, 731.8360066393614, 1.5939811033911373], [64, 1151, 1128.466224271798, -128588.81348484031, -227.90015459757157, 715.5016526421232, 1.5606916596909137], [54, 1156, 1148.5726620458995, -125814.42937593175, -219.07961687304015, 700.4481253919654, 1.5306109729163493], [59, 1176, 1166.961944162921, -122472.89823366192, -209.90041508425287, 685.2829227990027, 1.5019749145507262], [51, 1186, 1185.8114122404581, -118717.30206073177, -200.22965006961553, 669.7786351977905, 1.4730378409551832], [60, 1182, 1203.9021559263047, -114401.96790785609, -190.05193627190258, 650.7489088735505, 1.4410106799227245], [50, 1187, 1219.252235759979, -108373.65953890502, -177.7706964323982, 625.0622306098048, 1.4033659007851016], [54, 1208, 1235.3059949191388, -103776.44535246746, -168.01739128491886, 601.6735985325691, 1.3678840147182045], [78, 1247, 1253.5869690944037, -100427.21257941338, -160.22376596968363, 581.2044137220811, 1.3345759327951405], [61, 1321, 1273.595021893862, -95876.92939569919, -150.56109320076993, 558.174087267923, 1.2975533711981428], [88, 1420, 1308.5422963048659, -91440.31617761923, -139.7590531629485, 530.6416030589909, 1.2481386649201143], [70, 1862, 1399.2722750017035, -86630.53867037191, -123.82227564719875, 482.8358147514897, 1.1513429212724815], [60, 1804, 1515.6689728755591, -83167.88371089716, -109.74412645408886, 435.5787665251204, 1.0507205870097283], [59, 1756, 1619.6537389776506, -79313.1373580395, -97.93838701363802, 392.35119564423746, 0.9646782183078773], [64, 1693, 1709.8520717742715, -75042.7754339379, -87.77692137550572, 350.55529647920315, 0.8874728350717506], [52, 1635, 1787.62550343858, -71425.04872290586, -79.91052777611026, 314.9770178586767, 0.8227298107731542], [56, 1584, 1856.274143641366, -69018.3592692434, -74.36224816863884, 288.01044072418273, 0.7720388044941164], [74, 1532, 1912.3696124229427, -65701.19108220657, -68.71181246073469, 260.6564084803957, 0.7236096630146555], [114, 1461, 1947.9050194228269, -61393.1758181778, -63.035081491159026, 231.16393640836571, 0.6751996735769568], [131, 1398, 1954.9580709104546, -57303.86226840527, -58.6241343188685, 204.51566071354222, 0.6339436156717544], [441, 1338, 1920.0845665614802, -53003.56544505126, -55.20961562643143, 180.53710663190884, 0.6010064487032096]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 579090.1018226708, 1984.3895180566637, 794.1976618676379, 2.2863660720641907], [101, 504, 666.3114308957505, 654994.6039346324, 1966.0314188339694, 801.8915402948611, 2.1501821655920383], [65, 551, 721.1424425150965, 700409.8245314332, 1942.5006302184765, 810.4019333318075, 2.077761895110684], [53, 593, 757.4765311869324, 726574.1445434467, 1918.4070123068684, 815.8796867419061, 2.0341573335795933], [54, 620, 780.4182089417114, 739018.0882801425, 1893.9027301330918, 822.1314143978245, 2.011698981557118], [39, 648, 787.2394933018742, 735612.8196445594, 1868.8412507335477, 840.7289231948547, 2.025492451585069], [32, 674, 783.295106379799, 721929.4287311737, 1843.3140277557902, 863.7344179879951, 2.0581805868354692], [31, 705, 772.3184664651072, 702172.1661282271, 1818.34876833144, 882.8109481073785, 2.095519550380126], [34, 743, 755.5948864772723, 676274.2701226803, 1790.0445919521774, 907.4188431349253, 2.1479069486948172], [25, 770, 733.2801280101717, 644739.3560722482, 1758.5076465165157, 942.2270607916229, 2.221768985590072], [23, 798, 705.0823953200298, 608241.8258541685, 1725.3070843673345, 978.678444974414, 2.309171279904825], [46, 731, 679.6849616043089, 574093.7656300052, 1689.2937112362488, 1020.7037521045437, 2.4018843723335563], [38, 693, 661.2234869938007, 545357.8380806628, 1649.5416415411623, 1073.1591706385566, 2.4969739191314835], [40, 665, 645.7656636747749, 518561.99321640324, 1606.0376770901373, 1128.6049021662166, 2.5911321497962345], [24, 639, 631.3836409147973, 492622.8116804608, 1560.4547845639804, 1185.6431299460257, 2.6858787510025888], [27, 618, 623.6519062582148, 472442.43322562525, 1515.0837461884282, 1233.3592722467988, 2.756320694820169], [26, 607, 618.1991892236267, 451891.35768006, 1461.960369917578, 1288.3657278045728, 2.8295113316257474], [37, 605, 613.1480790457492, 431580.52109004033, 1407.7529909633413, 1337.7035520108868, 2.895031713116739], [40, 598, 605.9192816957346, 410948.6485813888, 1356.4468436498732, 1379.1350773602337, 2.9570051927779004], [21, 593, 594.614044958479, 388620.7694368255, 1307.1361927347757, 1409.6058752106946, 3.0177783629418573], [26, 605, 585.7606828381125, 366286.03881393553, 1250.6337470081328, 1437.2863518038662, 3.0702066869998097], [29, 602, 577.5147629228793, 344131.8366232376, 1191.768102624911, 1461.8529369654677, 3.1183608429800618], [33, 583, 571.6406726458698, 326225.1250204127, 1141.3642892499656, 1477.429085344917, 3.1509958617514235], [21, 570, 567.3196075776277, 310508.7400128223, 1094.6518888661349, 1487.1821593363147, 3.1733959115970896], [35, 579, 566.3049097540636, 296062.96903519187, 1045.5956285590632, 1494.8083790186467, 3.184371070011158], [26, 583, 564.4748875421996, 281138.07672059524, 996.1048150244865, 1498.7881402802714, 3.1937718041559697], [23, 583, 563.6774808520777, 270394.33250636276, 959.3937728277658, 1487.0189109094508, 3.1834569019613417], [31, 580, 564.080332543945, 259014.20464295542, 918.3592821782233, 1480.1177610754355, 3.174926878589568], [22, 585, 568.4578596050166, 247826.1144449741, 871.9243133243754, 1479.2362835386384, 3.161736763371161], [31, 585, 577.8133836442377, 237770.845197093, 823.0022077283332, 1482.5594840883643, 3.139556741809795], [30, 587, 583.890447182401, 226234.50715345488, 774.9210772163282, 1493.569002256613, 3.1347508743112242], [26, 588, 588.7332941603438, 214488.64727069048, 728.6445302081851, 1508.9178795228713, 3.1378311877652023], [30, 577, 593.3288656184958, 202999.86215278617, 684.2743507554643, 1522.9398467301141, 3.1401450359625707], [35, 593, 599.1144178234287, 190070.1051476771, 634.5035255142021, 1546.1946332480097, 3.148714372821247], [30, 596, 604.0168762582872, 176600.79963039325, 584.7545211795571, 1571.1128884385575, 3.161078133010358], [32, 592, 608.7374278541419, 161771.98393341136, 531.5000410067546, 1598.4175434141368, 3.176041639809634], [27, 601, 612.3128676974197, 144058.39661803457, 470.5385243977019, 1631.566384863241, 3.199423664600885], [42, 616, 615.8680691275756, 125165.46440896092, 406.4684327156867, 1665.8977676576399, 3.2235647684050734], [34, 627, 618.3707451800456, 103216.15749851275, 333.83260221490656, 1698.9680532185037, 3.248809228290445], [20, 629, 618.4372143951659, 79967.66718332618, 258.61208000406276, 1733.0907058814907, 3.2810957955653532], [30, 616, 619.7894474788376, 55025.287664437405, 177.56122789204545, 1764.1801822072935, 3.306781140761259], [29, 618, 621.1703968707503, 29917.36028218845, 96.32577609268617, 1781.4209543585068, 3.319204223367515], [24, 632, 624.4337280335847, 5200.38395836372, 16.656319877980778, 1786.8016845313803, 3.3155155682668016], [41, 640, 629.4098427218046, -15480.94780890317, -49.19194698944566, 1773.6209707200735, 3.290180422310677], [32, 644, 634.539318869226, -34126.932447449304, -107.56443748281774, 1746.5810604088797, 3.2517801492008664], [29, 642, 639.7491371710681, -50990.26837358493, -159.40707196280343, 1709.4617450597777, 3.2039144426579127], [27, 663, 647.2454425303501, -65692.06383754365, -202.98965283008005, 1656.6005024636322, 3.1356708245488156], [47, 669, 656.4827710203692, -76769.6429633882, -233.88166865084784, 1586.263367604898, 3.0467165552414204], [35, 664, 662.0507521335019, -80916.02683970236, -244.44055558866197, 1501.6100462225154, 2.951814163798141], [22, 680, 670.304518327072, -80244.3187812565, -239.42645942929974, 1393.7986280715616, 2.8263114261957165], [29, 694, 682.0072246014435, -73224.96716086917, -214.7336993494781, 1263.2980518182828, 2.667561845037861], [29, 708, 697.0073345999263, -64395.24704142512, -184.77638281493037, 1129.636697362568, 2.495208225828501], [44, 740, 714.6992438595885, -54871.66335039772, -153.55175990973328, 993.3453418647225, 2.3107051901402733], [40, 748, 731.6274379687737, -46233.842898913594, -126.38630127725439, 858.8932127643794, 2.1236388639957853], [33, 766, 749.827523794892, -37523.07749585744, -100.08455626155839, 726.2167107950241, 1.9288946185490463], [36, 796, 771.5340044686911, -28400.66001668654, -73.62127878276566, 589.8699596046911, 1.7137863329435834], [48, 819, 794.5241232990452, -19391.464168971605, -48.812776353356846, 451.3258881080688, 1.477228405874995], [34, 833, 818.010093419614, -11353.337525763001, -27.758428941387375, 308.1377855278013, 1.2029545487390985], [40, 843, 847.193014683848, -6004.944398737014, -14.17609516286655, 174.01586830068567, 0.8882992893651278], [40, 860, 874.9118881198643, -2846.5645901243543, -6.507088608068771, 49.62334849556987, 0.4667850580636025], [41, 890, 900.7436152744635, -2250.711662587046, -4.997452381388763, 13.491322241149065, 0.23987375460016266], [54, 900, 924.6538587617463, -2025.541110824947, -4.381187817758005, 12.783821516732514, 0.23046067386731578], [49, 906, 942.2651862480523, -1824.6502718914305, -3.872901808368604, 12.19677126479848, 0.2229933634841931], [52, 918, 958.0265479843835, -1653.4773958526744, -3.4518404512515186, 11.683744094676932, 0.2164503624768715], [49, 933, 972.0301994996937, -1502.539139592803, -3.091548267463631, 11.194740084381769, 0.2103406442206863], [55, 948, 982.6950901029222, -1362.7777925119017, -2.7735516463589365, 10.72028088224095, 0.20471504468184287], [59, 949, 988.5808782664636, -1223.5014614341126, -2.475268313058202, 10.150280409644534, 0.19860445608271252], [49, 963, 989.122185648845, -1082.7106102596933, -2.1892353158562634, 9.52054284334566, 0.19229232730283854], [49, 976, 986.0002242460553, -947.7944509449587, -1.9225035200568832, 8.876600727164178, 0.1859691505258136], [47, 994, 984.6273112360902, -825.8896024093177, -1.6775679345568935, 8.23032214670349, 0.17919609840337328], [43, 1020, 985.5537594129897, -730.3436699132553, -1.4820980853408925, 7.662971190544345, 0.17282816080077723], [41, 1039, 988.8460923441614, -657.5192629299321, -1.3298717930334638, 7.175897363106353, 0.16696668694222805], [50, 1029, 997.0593149933917, -580.8032145889193, -1.1650324225550588, 6.601893064880426, 0.15948866268443604], [53, 1045, 1006.5628540930174, -511.63770149355565, -1.0166035820079542, 6.042425757070596, 0.151859261377455], [53, 1051, 1017.1405338390672, -454.6832687742445, -0.8940421773539994, 5.546049316486426, 0.14472963074830972], [45, 1048, 1028.7149214037931, -407.4847142955956, -0.7922208686144816, 5.084077822417897, 0.13778903119014485], [48, 1044, 1042.5732809167782, -374.2743477445835, -0.7179818523940464, 4.788085878537738, 0.13282619801150347], [46, 1059, 1061.073689853978, -350.04926072988974, -0.6598019799700481, 4.570279178811609, 0.1286336708537136], [62, 1075, 1085.030100296678, -328.6461850027627, -0.6057826136121044, 4.329083504840714, 0.12380356759976449], [63, 1090, 1106.5207916475836, -317.19560325890706, -0.5733206382622241, 4.169318812809933, 0.12031196905800855], [64, 1151, 1128.466224271798, -306.7713945414167, -0.5436961921290592, 4.04067367906307, 0.1172839754156031], [54, 1156, 1148.5726620458995, -298.00577696977024, -0.5189149747634534, 3.9341680614154635, 0.11471053042699211], [59, 1176, 1166.961944162921, -299.31283109362795, -0.5129778783117549, 3.91398747996473, 0.11351086695396706], [51, 1186, 1185.8114122404581, -304.4826265474695, -0.5135430868761561, 3.911059037327626, 0.11256294429204221], [60, 1182, 1203.9021559263047, -307.82874621129844, -0.51138499037648, 3.912714087566298, 0.1117376497357607], [50, 1187, 1219.252235759979, -312.59110762263106, -0.5127587195733755, 3.940319795167894, 0.11142304588895659], [54, 1208, 1235.3059949191388, -314.4260738225938, -0.5090658915537372, 3.920692806121318, 0.11042062595295946], [78, 1247, 1253.5869690944037, -316.04818573025295, -0.5042301707372843, 3.88785365120444, 0.10915252614201387], [61, 1321, 1273.595021893862, -316.32163218018144, -0.4967381730336927, 3.87517153252044, 0.10811497705185102], [88, 1420, 1308.5422963048659, -304.9701012906381, -0.4661218856307963, 3.741118141014307, 0.10480039037492729], [70, 1862, 1399.2722750017035, -286.0307434276999, -0.40882785793401316, 3.452370266800767, 0.09735620591583363], [60, 1804, 1515.6689728755591, -266.20066087454626, -0.3512649076262408, 3.1407287004442233, 0.08922141421573956], [59, 1756, 1619.6537389776506, -244.7605327494802, -0.30223809800726503, 2.851569139512236, 0.08224072597942804], [64, 1693, 1709.8520717742715, -220.603035778466, -0.2580375687699716, 2.5691933144059904, 0.07597578456037091], [52, 1635, 1787.62550343858, -199.92601566869484, -0.22367773930739737, 2.3083033859706235, 0.07043106324929338], [56, 1584, 1856.274143641366, -178.54679382067667, -0.192371147798708, 2.035700023968529, 0.0649070550053875], [74, 1532, 1912.3696124229427, -157.34880469258368, -0.16455898867084087, 1.7862237999055746, 0.05990155412094109], [114, 1461, 1947.9050194228269, -134.75242823321406, -0.13835626161396905, 1.5333825982636828, 0.054991738316519066], [131, 1398, 1954.9580709104546, -114.6680609492478, -0.1173099951917077, 1.2772216523868973, 0.05009801101711326], [441, 1338, 1920.0845665614802, -98.906396787632, -0.10302295899888957, 1.0918463199804667, 0.046738702326131774]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}}, 'bounds': [0.27917221855045943, 0.5516964465803048], 'bestTools': ['DecisionTool2', 'DecisionTool1', 'DecisionTool2'], '_className': 'ConfidenceDecisionNode', '_version': 1}, 'state': {}, 'minCasesPerPartition': 70, 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 467826.4981191129, 1603.1184028439893, 982.1881055844433, 2.5426044342143435], [101, 504, 666.3114308957505, 528490.8031842152, 1586.3176847311258, 983.9435041094891, 2.381784808287103], [65, 551, 721.1424425150965, 566587.6812111209, 1571.3613505677413, 981.8584869816867, 2.2870201568194934], [53, 593, 757.4765311869324, 588438.156782675, 1553.6802331304395, 980.0049925071575, 2.229387822618676], [54, 620, 780.4182089417114, 598058.9204544935, 1532.6626508766194, 980.4135923027255, 2.196832961813439], [39, 648, 787.2394933018742, 595189.2995086145, 1512.0920750869486, 982.99257861464, 2.1901696389323777], [32, 674, 783.295106379799, 584650.681144051, 1492.7979924352276, 984.5659628102719, 2.197433664610511], [31, 705, 772.3184664651072, 568744.2484563492, 1472.8231245317365, 985.2064199678813, 2.2137138305004003], [34, 743, 755.5948864772723, 548442.8776997251, 1451.6849902377464, 985.6964857268364, 2.238634386103309], [25, 770, 733.2801280101717, 525382.706267886, 1432.965891748803, 987.0907337378834, 2.2740481234839773], [23, 798, 705.0823953200298, 498681.5183528668, 1414.5340223010965, 987.8226867193093, 2.3199340214250355], [46, 731, 679.6849616043089, 473633.6838748095, 1393.685929895692, 989.2505121642839, 2.3645874635475557], [38, 693, 661.2234869938007, 451653.2939665361, 1366.1138869096783, 995.1645783091144, 2.404525517584991], [40, 665, 645.7656636747749, 432653.36991052685, 1339.9701911943798, 1001.9207839183987, 2.441379478029498], [24, 639, 631.3836409147973, 415500.70557218353, 1316.1592371008348, 1007.9690372259919, 2.476469619663136], [27, 618, 623.6519062582148, 402465.5586253533, 1290.6737062348298, 1013.8865998423411, 2.499076990401595], [26, 607, 618.1991892236267, 389261.09536828665, 1259.3387443847837, 1024.7349827706264, 2.5234670426113603], [37, 605, 613.1480790457492, 375796.3208274763, 1225.7930300045407, 1033.1961436571785, 2.544279235912517], [40, 598, 605.9192816957346, 361413.5670403987, 1192.9429478756358, 1043.4801953459466, 2.5721174174775183], [21, 593, 594.614044958479, 344399.7735477707, 1158.397708455809, 1047.862698922668, 2.6019004740857268], [26, 605, 585.7606828381125, 327674.96602411446, 1118.8015024718702, 1048.905316032369, 2.6227935423084596], [29, 602, 577.5147629228793, 311993.0060445265, 1080.4676384911384, 1047.583031697504, 2.639786197418429], [33, 583, 571.6406726458698, 300351.63411547506, 1050.8406713793868, 1040.2590377434285, 2.644023186756073], [21, 570, 567.3196075776277, 289996.6168048502, 1022.3394817714607, 1028.5211109447323, 2.639057055224421], [35, 579, 566.3049097540636, 280054.13229566737, 989.0577583630346, 1015.7990027303528, 2.6250331907656013], [26, 583, 564.4748875421996, 269415.4618561862, 954.5702308538748, 1001.1200460164004, 2.6102183429752492], [23, 583, 563.6774808520777, 262037.12118649483, 929.7413151591187, 980.5795145693014, 2.5851285079122257], [31, 580, 564.080332543945, 256038.50452377542, 907.8086568594504, 964.0925117557081, 2.562388327619304], [22, 585, 568.4578596050166, 252014.84753109206, 886.6614939802236, 949.0063409865721, 2.532453564111772], [31, 585, 577.8133836442377, 248931.0482763152, 861.631299387081, 937.4368966797739, 2.49650990995888], [30, 587, 583.890447182401, 244790.8083623263, 838.4819773763359, 930.9642464701626, 2.474895622580027], [26, 588, 588.7332941603438, 240378.78079377607, 816.596524022329, 928.9921367769045, 2.4620835810618735], [30, 577, 593.3288656184958, 235536.43598308324, 793.949020961777, 932.9649826612184, 2.457768673297603], [35, 593, 599.1144178234287, 230494.56677397917, 769.4509092649166, 943.4407064593674, 2.459566058097794], [30, 596, 604.0168762582872, 226159.54573746584, 748.8517444693265, 954.2564264250507, 2.463565318405578], [32, 592, 608.7374278541419, 221760.42607211822, 728.5913956493364, 968.5041426544662, 2.4722467250018108], [27, 601, 612.3128676974197, 214981.59918010086, 702.1952682083306, 985.9859259686688, 2.4871657937543765], [42, 616, 615.8680691275756, 206908.0703074762, 671.9233572234955, 1002.0943754879978, 2.500152729613687], [34, 627, 618.3707451800456, 197230.31106423997, 637.9031110432437, 1010.8706595904476, 2.505990379924332], [20, 629, 618.4372143951659, 188064.72417361918, 608.1934262560419, 1021.3655439800741, 2.51883004540106], [30, 616, 619.7894474788376, 177899.49697519335, 574.0642977993512, 1035.7689439546082, 2.533759704623051], [29, 618, 621.1703968707503, 166196.271787549, 535.1068647984208, 1050.636949761916, 2.549042235188664], [24, 632, 624.4337280335847, 153294.2220726125, 490.98636153224476, 1068.9840724434248, 2.5644752505411077], [41, 640, 629.4098427218046, 140536.38032981675, 446.5655628201957, 1086.1740894908082, 2.574773469871046], [32, 644, 634.539318869226, 126730.03639650389, 399.439507144622, 1103.6411155715418, 2.584882093820425], [29, 642, 639.7491371710681, 111900.09151730503, 349.8249079697722, 1119.231279228897, 2.5924544614604237], [27, 663, 647.2454425303501, 96008.23885518893, 296.667176148365, 1132.6382819920025, 2.5927890800267797], [47, 669, 656.4827710203692, 80018.982484126, 243.7809064196848, 1140.681042408093, 2.583607382716839], [35, 664, 662.0507521335019, 63307.23640573951, 191.2458711110222, 1149.6858627916233, 2.582854996589537], [22, 680, 670.304518327072, 45210.38674241446, 134.8950678573653, 1152.4755598024128, 2.5700161948760254], [29, 694, 682.0072246014435, 29427.989708740246, 86.2981758761796, 1144.9890768167488, 2.5395820893522374], [29, 708, 697.0073345999263, 15095.865671981548, 43.31623190348891, 1130.9627478680466, 2.4966723256078565], [44, 740, 714.6992438595885, 2347.880837814503, 6.570262548859708, 1107.5136519951589, 2.4398827368246563], [40, 748, 731.6274379687737, -10839.152799762986, -29.630252331311848, 1081.6125253859916, 2.383125606504419], [33, 766, 749.827523794892, -23663.01367264134, -63.11588444468497, 1055.4735520155316, 2.325407452733299], [36, 796, 771.5340044686911, -35436.17430297201, -91.85900841110629, 1026.1533053738342, 2.260396674197986], [48, 819, 794.5241232990452, -45646.33711114669, -114.90233152799112, 996.3497232826718, 2.1948681160633265], [34, 833, 818.010093419614, -56185.87099055746, -137.37207264931837, 971.3039878519611, 2.1357692333699165], [40, 843, 847.193014683848, -66247.11567305274, -156.39202525241447, 950.8268097504587, 2.076421884959117], [40, 860, 874.9118881198643, -75591.96772642512, -172.7990412585842, 936.4091621882596, 2.0277141868069837], [41, 890, 900.7436152744635, -87351.69600789635, -193.95462710280685, 928.5841500444686, 1.990059715247872], [54, 900, 924.6538587617463, -100286.37988653741, -216.9165876208775, 926.8120592273824, 1.9622860486844365], [49, 906, 942.2651862480523, -113207.61127880005, -240.2882180749444, 931.6714037305406, 1.9489508252305945], [52, 918, 958.0265479843835, -124215.54968756462, -259.31546458478607, 934.9244762825593, 1.9362238448406883], [49, 933, 972.0301994996937, -135062.36575006542, -277.8974682465264, 938.48780435743, 1.9258857024456595], [55, 948, 982.6950901029222, -145348.1116388483, -295.8152800450551, 943.8754383887951, 1.9208967456341817], [59, 949, 988.5808782664636, -154256.8502912457, -312.07734983048516, 946.8174962197812, 1.9181523833853813], [49, 963, 989.122185648845, -160653.81202853814, -324.8411861738848, 948.6834844433675, 1.9195161474435625], [49, 976, 986.0002242460553, -165108.73034262026, -334.9060705718816, 949.2474436157664, 1.923123977200898], [47, 994, 984.6273112360902, -168196.46206225042, -341.644925227797, 944.2462219394298, 1.9193879339183886], [43, 1020, 985.5537594129897, -167578.5446045452, -340.0698196399909, 932.576441850336, 1.9065936054309776], [41, 1039, 988.8460923441614, -164635.92830396976, -332.9859511578457, 914.5315477256437, 1.884911943450251], [50, 1029, 997.0593149933917, -161375.17509093462, -323.7022565543238, 892.2953106178265, 1.854171404694584], [53, 1045, 1006.5628540930174, -158723.01371965124, -315.3762590666464, 871.7975409154251, 1.824078151841528], [53, 1051, 1017.1405338390672, -154576.75817184622, -303.9437580732644, 849.8834835448015, 1.7916174560804736], [45, 1048, 1028.7149214037931, -150051.40252755745, -291.72591824136475, 828.3907551395513, 1.7588393502984483], [48, 1044, 1042.5732809167782, -144959.3098066014, -278.07984812181763, 807.0859922294462, 1.7244979440054986], [46, 1059, 1061.073689853978, -139812.68745392922, -263.5305894224365, 782.636062113168, 1.683306599207149], [62, 1075, 1085.030100296678, -135614.4811183681, -249.97367553450775, 755.7472138909835, 1.6357745824443095], [63, 1090, 1106.5207916475836, -131106.66390561414, -236.97099032436506, 731.8360066393614, 1.5939811033911373], [64, 1151, 1128.466224271798, -128588.81348484031, -227.90015459757157, 715.5016526421232, 1.5606916596909137], [54, 1156, 1148.5726620458995, -125814.42937593175, -219.07961687304015, 700.4481253919654, 1.5306109729163493], [59, 1176, 1166.961944162921, -122472.89823366192, -209.90041508425287, 685.2829227990027, 1.5019749145507262], [51, 1186, 1185.8114122404581, -118717.30206073177, -200.22965006961553, 669.7786351977905, 1.4730378409551832], [60, 1182, 1203.9021559263047, -114401.96790785609, -190.05193627190258, 650.7489088735505, 1.4410106799227245], [50, 1187, 1219.252235759979, -108373.65953890502, -177.7706964323982, 625.0622306098048, 1.4033659007851016], [54, 1208, 1235.3059949191388, -103776.44535246746, -168.01739128491886, 601.6735985325691, 1.3678840147182045], [78, 1247, 1253.5869690944037, -100427.21257941338, -160.22376596968363, 581.2044137220811, 1.3345759327951405], [61, 1321, 1273.595021893862, -95876.92939569919, -150.56109320076993, 558.174087267923, 1.2975533711981428], [88, 1420, 1308.5422963048659, -91440.31617761923, -139.7590531629485, 530.6416030589909, 1.2481386649201143], [70, 1862, 1399.2722750017035, -86630.53867037191, -123.82227564719875, 482.8358147514897, 1.1513429212724815], [60, 1804, 1515.6689728755591, -83167.88371089716, -109.74412645408886, 435.5787665251204, 1.0507205870097283], [59, 1756, 1619.6537389776506, -79313.1373580395, -97.93838701363802, 392.35119564423746, 0.9646782183078773], [64, 1693, 1709.8520717742715, -75042.7754339379, -87.77692137550572, 350.55529647920315, 0.8874728350717506], [52, 1635, 1787.62550343858, -71425.04872290586, -79.91052777611026, 314.9770178586767, 0.8227298107731542], [56, 1584, 1856.274143641366, -69018.3592692434, -74.36224816863884, 288.01044072418273, 0.7720388044941164], [74, 1532, 1912.3696124229427, -65701.19108220657, -68.71181246073469, 260.6564084803957, 0.7236096630146555], [114, 1461, 1947.9050194228269, -61393.1758181778, -63.035081491159026, 231.16393640836571, 0.6751996735769568], [131, 1398, 1954.9580709104546, -57303.86226840527, -58.6241343188685, 204.51566071354222, 0.6339436156717544], [441, 1338, 1920.0845665614802, -53003.56544505126, -55.20961562643143, 180.53710663190884, 0.6010064487032096]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 579090.1018226708, 1984.3895180566637, 794.1976618676379, 2.2863660720641907], [101, 504, 666.3114308957505, 654994.6039346324, 1966.0314188339694, 801.8915402948611, 2.1501821655920383], [65, 551, 721.1424425150965, 700409.8245314332, 1942.5006302184765, 810.4019333318075, 2.077761895110684], [53, 593, 757.4765311869324, 726574.1445434467, 1918.4070123068684, 815.8796867419061, 2.0341573335795933], [54, 620, 780.4182089417114, 739018.0882801425, 1893.9027301330918, 822.1314143978245, 2.011698981557118], [39, 648, 787.2394933018742, 735612.8196445594, 1868.8412507335477, 840.7289231948547, 2.025492451585069], [32, 674, 783.295106379799, 721929.4287311737, 1843.3140277557902, 863.7344179879951, 2.0581805868354692], [31, 705, 772.3184664651072, 702172.1661282271, 1818.34876833144, 882.8109481073785, 2.095519550380126], [34, 743, 755.5948864772723, 676274.2701226803, 1790.0445919521774, 907.4188431349253, 2.1479069486948172], [25, 770, 733.2801280101717, 644739.3560722482, 1758.5076465165157, 942.2270607916229, 2.221768985590072], [23, 798, 705.0823953200298, 608241.8258541685, 1725.3070843673345, 978.678444974414, 2.309171279904825], [46, 731, 679.6849616043089, 574093.7656300052, 1689.2937112362488, 1020.7037521045437, 2.4018843723335563], [38, 693, 661.2234869938007, 545357.8380806628, 1649.5416415411623, 1073.1591706385566, 2.4969739191314835], [40, 665, 645.7656636747749, 518561.99321640324, 1606.0376770901373, 1128.6049021662166, 2.5911321497962345], [24, 639, 631.3836409147973, 492622.8116804608, 1560.4547845639804, 1185.6431299460257, 2.6858787510025888], [27, 618, 623.6519062582148, 472442.43322562525, 1515.0837461884282, 1233.3592722467988, 2.756320694820169], [26, 607, 618.1991892236267, 451891.35768006, 1461.960369917578, 1288.3657278045728, 2.8295113316257474], [37, 605, 613.1480790457492, 431580.52109004033, 1407.7529909633413, 1337.7035520108868, 2.895031713116739], [40, 598, 605.9192816957346, 410948.6485813888, 1356.4468436498732, 1379.1350773602337, 2.9570051927779004], [21, 593, 594.614044958479, 388620.7694368255, 1307.1361927347757, 1409.6058752106946, 3.0177783629418573], [26, 605, 585.7606828381125, 366286.03881393553, 1250.6337470081328, 1437.2863518038662, 3.0702066869998097], [29, 602, 577.5147629228793, 344131.8366232376, 1191.768102624911, 1461.8529369654677, 3.1183608429800618], [33, 583, 571.6406726458698, 326225.1250204127, 1141.3642892499656, 1477.429085344917, 3.1509958617514235], [21, 570, 567.3196075776277, 310508.7400128223, 1094.6518888661349, 1487.1821593363147, 3.1733959115970896], [35, 579, 566.3049097540636, 296062.96903519187, 1045.5956285590632, 1494.8083790186467, 3.184371070011158], [26, 583, 564.4748875421996, 281138.07672059524, 996.1048150244865, 1498.7881402802714, 3.1937718041559697], [23, 583, 563.6774808520777, 270394.33250636276, 959.3937728277658, 1487.0189109094508, 3.1834569019613417], [31, 580, 564.080332543945, 259014.20464295542, 918.3592821782233, 1480.1177610754355, 3.174926878589568], [22, 585, 568.4578596050166, 247826.1144449741, 871.9243133243754, 1479.2362835386384, 3.161736763371161], [31, 585, 577.8133836442377, 237770.845197093, 823.0022077283332, 1482.5594840883643, 3.139556741809795], [30, 587, 583.890447182401, 226234.50715345488, 774.9210772163282, 1493.569002256613, 3.1347508743112242], [26, 588, 588.7332941603438, 214488.64727069048, 728.6445302081851, 1508.9178795228713, 3.1378311877652023], [30, 577, 593.3288656184958, 202999.86215278617, 684.2743507554643, 1522.9398467301141, 3.1401450359625707], [35, 593, 599.1144178234287, 190070.1051476771, 634.5035255142021, 1546.1946332480097, 3.148714372821247], [30, 596, 604.0168762582872, 176600.79963039325, 584.7545211795571, 1571.1128884385575, 3.161078133010358], [32, 592, 608.7374278541419, 161771.98393341136, 531.5000410067546, 1598.4175434141368, 3.176041639809634], [27, 601, 612.3128676974197, 144058.39661803457, 470.5385243977019, 1631.566384863241, 3.199423664600885], [42, 616, 615.8680691275756, 125165.46440896092, 406.4684327156867, 1665.8977676576399, 3.2235647684050734], [34, 627, 618.3707451800456, 103216.15749851275, 333.83260221490656, 1698.9680532185037, 3.248809228290445], [20, 629, 618.4372143951659, 79967.66718332618, 258.61208000406276, 1733.0907058814907, 3.2810957955653532], [30, 616, 619.7894474788376, 55025.287664437405, 177.56122789204545, 1764.1801822072935, 3.306781140761259], [29, 618, 621.1703968707503, 29917.36028218845, 96.32577609268617, 1781.4209543585068, 3.319204223367515], [24, 632, 624.4337280335847, 5200.38395836372, 16.656319877980778, 1786.8016845313803, 3.3155155682668016], [41, 640, 629.4098427218046, -15480.94780890317, -49.19194698944566, 1773.6209707200735, 3.290180422310677], [32, 644, 634.539318869226, -34126.932447449304, -107.56443748281774, 1746.5810604088797, 3.2517801492008664], [29, 642, 639.7491371710681, -50990.26837358493, -159.40707196280343, 1709.4617450597777, 3.2039144426579127], [27, 663, 647.2454425303501, -65692.06383754365, -202.98965283008005, 1656.6005024636322, 3.1356708245488156], [47, 669, 656.4827710203692, -76769.6429633882, -233.88166865084784, 1586.263367604898, 3.0467165552414204], [35, 664, 662.0507521335019, -80916.02683970236, -244.44055558866197, 1501.6100462225154, 2.951814163798141], [22, 680, 670.304518327072, -80244.3187812565, -239.42645942929974, 1393.7986280715616, 2.8263114261957165], [29, 694, 682.0072246014435, -73224.96716086917, -214.7336993494781, 1263.2980518182828, 2.667561845037861], [29, 708, 697.0073345999263, -64395.24704142512, -184.77638281493037, 1129.636697362568, 2.495208225828501], [44, 740, 714.6992438595885, -54871.66335039772, -153.55175990973328, 993.3453418647225, 2.3107051901402733], [40, 748, 731.6274379687737, -46233.842898913594, -126.38630127725439, 858.8932127643794, 2.1236388639957853], [33, 766, 749.827523794892, -37523.07749585744, -100.08455626155839, 726.2167107950241, 1.9288946185490463], [36, 796, 771.5340044686911, -28400.66001668654, -73.62127878276566, 589.8699596046911, 1.7137863329435834], [48, 819, 794.5241232990452, -19391.464168971605, -48.812776353356846, 451.3258881080688, 1.477228405874995], [34, 833, 818.010093419614, -11353.337525763001, -27.758428941387375, 308.1377855278013, 1.2029545487390985], [40, 843, 847.193014683848, -6004.944398737014, -14.17609516286655, 174.01586830068567, 0.8882992893651278], [40, 860, 874.9118881198643, -2846.5645901243543, -6.507088608068771, 49.62334849556987, 0.4667850580636025], [41, 890, 900.7436152744635, -2250.711662587046, -4.997452381388763, 13.491322241149065, 0.23987375460016266], [54, 900, 924.6538587617463, -2025.541110824947, -4.381187817758005, 12.783821516732514, 0.23046067386731578], [49, 906, 942.2651862480523, -1824.6502718914305, -3.872901808368604, 12.19677126479848, 0.2229933634841931], [52, 918, 958.0265479843835, -1653.4773958526744, -3.4518404512515186, 11.683744094676932, 0.2164503624768715], [49, 933, 972.0301994996937, -1502.539139592803, -3.091548267463631, 11.194740084381769, 0.2103406442206863], [55, 948, 982.6950901029222, -1362.7777925119017, -2.7735516463589365, 10.72028088224095, 0.20471504468184287], [59, 949, 988.5808782664636, -1223.5014614341126, -2.475268313058202, 10.150280409644534, 0.19860445608271252], [49, 963, 989.122185648845, -1082.7106102596933, -2.1892353158562634, 9.52054284334566, 0.19229232730283854], [49, 976, 986.0002242460553, -947.7944509449587, -1.9225035200568832, 8.876600727164178, 0.1859691505258136], [47, 994, 984.6273112360902, -825.8896024093177, -1.6775679345568935, 8.23032214670349, 0.17919609840337328], [43, 1020, 985.5537594129897, -730.3436699132553, -1.4820980853408925, 7.662971190544345, 0.17282816080077723], [41, 1039, 988.8460923441614, -657.5192629299321, -1.3298717930334638, 7.175897363106353, 0.16696668694222805], [50, 1029, 997.0593149933917, -580.8032145889193, -1.1650324225550588, 6.601893064880426, 0.15948866268443604], [53, 1045, 1006.5628540930174, -511.63770149355565, -1.0166035820079542, 6.042425757070596, 0.151859261377455], [53, 1051, 1017.1405338390672, -454.6832687742445, -0.8940421773539994, 5.546049316486426, 0.14472963074830972], [45, 1048, 1028.7149214037931, -407.4847142955956, -0.7922208686144816, 5.084077822417897, 0.13778903119014485], [48, 1044, 1042.5732809167782, -374.2743477445835, -0.7179818523940464, 4.788085878537738, 0.13282619801150347], [46, 1059, 1061.073689853978, -350.04926072988974, -0.6598019799700481, 4.570279178811609, 0.1286336708537136], [62, 1075, 1085.030100296678, -328.6461850027627, -0.6057826136121044, 4.329083504840714, 0.12380356759976449], [63, 1090, 1106.5207916475836, -317.19560325890706, -0.5733206382622241, 4.169318812809933, 0.12031196905800855], [64, 1151, 1128.466224271798, -306.7713945414167, -0.5436961921290592, 4.04067367906307, 0.1172839754156031], [54, 1156, 1148.5726620458995, -298.00577696977024, -0.5189149747634534, 3.9341680614154635, 0.11471053042699211], [59, 1176, 1166.961944162921, -299.31283109362795, -0.5129778783117549, 3.91398747996473, 0.11351086695396706], [51, 1186, 1185.8114122404581, -304.4826265474695, -0.5135430868761561, 3.911059037327626, 0.11256294429204221], [60, 1182, 1203.9021559263047, -307.82874621129844, -0.51138499037648, 3.912714087566298, 0.1117376497357607], [50, 1187, 1219.252235759979, -312.59110762263106, -0.5127587195733755, 3.940319795167894, 0.11142304588895659], [54, 1208, 1235.3059949191388, -314.4260738225938, -0.5090658915537372, 3.920692806121318, 0.11042062595295946], [78, 1247, 1253.5869690944037, -316.04818573025295, -0.5042301707372843, 3.88785365120444, 0.10915252614201387], [61, 1321, 1273.595021893862, -316.32163218018144, -0.4967381730336927, 3.87517153252044, 0.10811497705185102], [88, 1420, 1308.5422963048659, -304.9701012906381, -0.4661218856307963, 3.741118141014307, 0.10480039037492729], [70, 1862, 1399.2722750017035, -286.0307434276999, -0.40882785793401316, 3.452370266800767, 0.09735620591583363], [60, 1804, 1515.6689728755591, -266.20066087454626, -0.3512649076262408, 3.1407287004442233, 0.08922141421573956], [59, 1756, 1619.6537389776506, -244.7605327494802, -0.30223809800726503, 2.851569139512236, 0.08224072597942804], [64, 1693, 1709.8520717742715, -220.603035778466, -0.2580375687699716, 2.5691933144059904, 0.07597578456037091], [52, 1635, 1787.62550343858, -199.92601566869484, -0.22367773930739737, 2.3083033859706235, 0.07043106324929338], [56, 1584, 1856.274143641366, -178.54679382067667, -0.192371147798708, 2.035700023968529, 0.0649070550053875], [74, 1532, 1912.3696124229427, -157.34880469258368, -0.16455898867084087, 1.7862237999055746, 0.05990155412094109], [114, 1461, 1947.9050194228269, -134.75242823321406, -0.13835626161396905, 1.5333825982636828, 0.054991738316519066], [131, 1398, 1954.9580709104546, -114.6680609492478, -0.1173099951917077, 1.2772216523868973, 0.05009801101711326], [441, 1338, 1920.0845665614802, -98.906396787632, -0.10302295899888957, 1.0918463199804667, 0.046738702326131774]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}, 'CompositeDecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[28, 474, 583.6455963441902, 579090.1018226708, 1984.3895180566637, 794.1976618676379, 2.2863660720641907], [101, 504, 666.3114308957505, 654994.6039346324, 1966.0314188339694, 801.8915402948611, 2.1501821655920383], [65, 551, 721.1424425150965, 700409.8245314332, 1942.5006302184765, 810.4019333318075, 2.077761895110684], [53, 593, 757.4765311869324, 726574.1445434467, 1918.4070123068684, 815.8796867419061, 2.0341573335795933], [54, 620, 780.4182089417114, 739018.0882801425, 1893.9027301330918, 822.1314143978245, 2.011698981557118], [39, 648, 787.2394933018742, 735612.8196445594, 1868.8412507335477, 840.7289231948547, 2.025492451585069], [32, 674, 783.295106379799, 721929.4287311737, 1843.3140277557902, 863.7344179879951, 2.0581805868354692], [31, 705, 772.3184664651072, 702172.1661282271, 1818.34876833144, 882.8109481073785, 2.095519550380126], [34, 743, 755.5948864772723, 676274.2701226803, 1790.0445919521774, 907.4188431349253, 2.1479069486948172], [25, 770, 733.2801280101717, 644739.3560722482, 1758.5076465165157, 942.2270607916229, 2.221768985590072], [23, 798, 705.0823953200298, 608241.8258541685, 1725.3070843673345, 978.678444974414, 2.309171279904825], [46, 731, 679.6849616043089, 574093.7656300052, 1689.2937112362488, 1020.7037521045437, 2.4018843723335563], [38, 693, 661.2234869938007, 545357.8380806628, 1649.5416415411623, 1073.1591706385566, 2.4969739191314835], [40, 665, 645.7656636747749, 518561.99321640324, 1606.0376770901373, 1128.6049021662166, 2.5911321497962345], [24, 639, 631.3836409147973, 492622.8116804608, 1560.4547845639804, 1185.6431299460257, 2.6858787510025888], [27, 618, 623.6519062582148, 472442.43322562525, 1515.0837461884282, 1233.3592722467988, 2.756320694820169], [26, 607, 618.1991892236267, 451891.35768006, 1461.960369917578, 1288.3657278045728, 2.8295113316257474], [37, 605, 613.1480790457492, 431580.52109004033, 1407.7529909633413, 1337.7035520108868, 2.895031713116739], [40, 598, 605.9192816957346, 411043.39979138767, 1356.7595955719896, 1378.4720995626571, 2.956294361589829], [21, 593, 594.614044958479, 389558.525949527, 1310.2903614620448, 1402.9353993307996, 3.010629595598688], [26, 605, 585.7606828381125, 367652.97778979305, 1255.3009737985496, 1423.6888207060279, 3.055649240351985], [29, 602, 577.5147629228793, 346228.47440179705, 1199.029000226725, 1436.6736289250164, 3.0913884937967664], [33, 583, 571.6406726458698, 330029.58356319624, 1154.6749535355364, 1430.696901068079, 3.1007612552707235], [21, 570, 567.3196075776277, 316404.24890471535, 1115.4356192824553, 1414.8407472480733, 3.0952515797587936], [35, 579, 566.3049097540636, 304429.65154135623, 1075.1439597215033, 1390.0103559904308, 3.0707179370795123], [26, 583, 564.4748875421996, 291942.4488436979, 1034.385958655681, 1356.469365005221, 3.0383566665289448], [23, 583, 563.6774808520777, 284251.9850997775, 1008.5625016280258, 1298.4859826067363, 2.974810883766543], [31, 580, 564.080332543945, 276338.1686186047, 979.7830297409861, 1240.379698726598, 2.9064504091382806], [22, 585, 568.4578596050166, 269318.84226312925, 947.541977694744, 1181.6829580843175, 2.8259041014841744], [31, 585, 577.8133836442377, 262958.46411403886, 910.1847466930382, 1130.1734557885052, 2.741162767598642], [30, 587, 583.890447182401, 256455.28539178846, 878.4363115695046, 1082.7329505387202, 2.6690152992561296], [26, 588, 588.7332941603438, 250278.56899088225, 850.2273320479745, 1041.305526817606, 2.6066688354308343], [30, 577, 593.3288656184958, 243590.88654041168, 821.0990587369739, 1012.2890774628572, 2.560121684162785], [35, 593, 599.1144178234287, 237009.59463621012, 791.1997694772945, 993.7460033800515, 2.524287887106063], [30, 596, 604.0168762582872, 231074.46182291023, 765.1258463318138, 979.976684315351, 2.496545044836994], [32, 592, 608.7374278541419, 224956.60902762326, 739.0924189452816, 976.9862832444221, 2.483049069433025], [27, 601, 612.3128676974197, 216059.71372273256, 705.7167181059489, 989.2178765056466, 2.491238783107415], [42, 616, 615.8680691275756, 206996.66321736405, 672.2110581592278, 1002.0963233812415, 2.5001551595386338], [34, 627, 618.3707451800456, 197230.31106423997, 637.9031110432437, 1010.8706595904476, 2.505990379924332], [20, 629, 618.4372143951659, 188064.72417361918, 608.1934262560419, 1021.3655439800741, 2.51883004540106], [30, 616, 619.7894474788376, 177899.49697519335, 574.0642977993512, 1035.7689439546082, 2.533759704623051], [29, 618, 621.1703968707503, 166196.271787549, 535.1068647984208, 1050.636949761916, 2.549042235188664], [24, 632, 624.4337280335847, 153294.2220726125, 490.98636153224476, 1068.9840724434248, 2.5644752505411077], [41, 640, 629.4098427218046, 140536.38032981675, 446.5655628201957, 1086.1740894908082, 2.574773469871046], [32, 644, 634.539318869226, 126730.03639650389, 399.439507144622, 1103.6411155715418, 2.584882093820425], [29, 642, 639.7491371710681, 112037.20587285665, 350.25355835031957, 1118.2925882452657, 2.591367097244393], [27, 663, 647.2454425303501, 97154.90097699023, 300.21038262446945, 1124.1837641774114, 2.5830940865635954], [47, 669, 656.4827710203692, 82631.34029703206, 251.73955492723263, 1120.5183101392247, 2.560671593921278], [35, 664, 662.0507521335019, 67846.46653855556, 204.95850603572575, 1115.4669469523042, 2.544126974728118], [22, 680, 670.304518327072, 52508.3892406882, 156.67025301198098, 1099.9236956021382, 2.5107373139071885], [29, 694, 682.0072246014435, 39769.94440336248, 116.62616749141783, 1071.0183118910315, 2.4561791018685915], [29, 708, 697.0073345999263, 28776.75047721301, 82.57230318452966, 1032.7140431762512, 2.38576376717756], [44, 740, 714.6992438595885, 20048.605891268657, 56.103615789489915, 980.6428449561719, 2.2958834742789915], [40, 748, 731.6274379687737, 12042.565594648078, 32.91993976628871, 918.1290859480333, 2.19564919128099], [33, 766, 749.827523794892, 5199.553018200617, 13.868664067932784, 847.7314591699031, 2.0840327159567686], [36, 796, 771.5340044686911, 710.0285195570777, 1.8405631260440205, 764.7846410942701, 1.951408148837163], [48, 819, 794.5241232990452, -1970.238681617983, -4.959544018467565, 678.1930185047637, 1.8108362710727093], [34, 833, 818.010093419614, -3782.3235702354186, -9.247620782828662, 594.0486010134571, 1.6702740309541293], [40, 843, 847.193014683848, -4241.910370876361, -10.014035284413529, 511.3728642122335, 1.522767949996783], [40, 860, 874.9118881198643, -3694.5450320092805, -8.445524817244504, 432.25222223411924, 1.3776615255010636], [41, 890, 900.7436152744635, -3580.3897517656783, -7.949853190299231, 355.77254665889154, 1.2318042308416826], [54, 900, 924.6538587617463, -3235.849633575456, -6.999050732149123, 279.31308994548124, 1.0772391357447044], [49, 906, 942.2651862480523, -3107.0805582155567, -6.594917446939644, 207.80193227128638, 0.9204374686051009], [52, 918, 958.0265479843835, -2655.052408560773, -5.542753307091136, 142.59685222826727, 0.7561749194042666], [49, 933, 972.0301994996937, -1795.3231962858179, -3.693965881327298, 69.82308560488352, 0.5253100515851964], [55, 948, 982.6950901029222, -1362.7777925119017, -2.7735516463589365, 10.72028088224095, 0.20471504468184287], [59, 949, 988.5808782664636, -1223.5014614341126, -2.475268313058202, 10.150280409644534, 0.19860445608271252], [49, 963, 989.122185648845, -1082.7106102596933, -2.1892353158562634, 9.52054284334566, 0.19229232730283854], [49, 976, 986.0002242460553, -947.7944509449587, -1.9225035200568832, 8.876600727164178, 0.1859691505258136], [47, 994, 984.6273112360902, -825.8896024093177, -1.6775679345568935, 8.23032214670349, 0.17919609840337328], [43, 1020, 985.5537594129897, -730.3436699132553, -1.4820980853408925, 7.662971190544345, 0.17282816080077723], [41, 1039, 988.8460923441614, -657.5192629299321, -1.3298717930334638, 7.175897363106353, 0.16696668694222805], [50, 1029, 997.0593149933917, -580.8032145889193, -1.1650324225550588, 6.601893064880426, 0.15948866268443604], [53, 1045, 1006.5628540930174, -511.63770149355565, -1.0166035820079542, 6.042425757070596, 0.151859261377455], [53, 1051, 1017.1405338390672, -454.6832687742445, -0.8940421773539994, 5.546049316486426, 0.14472963074830972], [45, 1048, 1028.7149214037931, -407.4847142955956, -0.7922208686144816, 5.084077822417897, 0.13778903119014485], [48, 1044, 1042.5732809167782, -374.2743477445835, -0.7179818523940464, 4.788085878537738, 0.13282619801150347], [46, 1059, 1061.073689853978, -350.04926072988974, -0.6598019799700481, 4.570279178811609, 0.1286336708537136], [62, 1075, 1085.030100296678, -328.6461850027627, -0.6057826136121044, 4.329083504840714, 0.12380356759976449], [63, 1090, 1106.5207916475836, -317.19560325890706, -0.5733206382622241, 4.169318812809933, 0.12031196905800855], [64, 1151, 1128.466224271798, -306.7713945414167, -0.5436961921290592, 4.04067367906307, 0.1172839754156031], [54, 1156, 1148.5726620458995, -298.00577696977024, -0.5189149747634534, 3.9341680614154635, 0.11471053042699211], [59, 1176, 1166.961944162921, -299.31283109362795, -0.5129778783117549, 3.91398747996473, 0.11351086695396706], [51, 1186, 1185.8114122404581, -304.4826265474695, -0.5135430868761561, 3.911059037327626, 0.11256294429204221], [60, 1182, 1203.9021559263047, -307.82874621129844, -0.51138499037648, 3.912714087566298, 0.1117376497357607], [50, 1187, 1219.252235759979, -312.59110762263106, -0.5127587195733755, 3.940319795167894, 0.11142304588895659], [54, 1208, 1235.3059949191388, -314.4260738225938, -0.5090658915537372, 3.920692806121318, 0.11042062595295946], [78, 1247, 1253.5869690944037, -316.04818573025295, -0.5042301707372843, 3.88785365120444, 0.10915252614201387], [61, 1321, 1273.595021893862, -316.32163218018144, -0.4967381730336927, 3.87517153252044, 0.10811497705185102], [88, 1420, 1308.5422963048659, -304.9701012906381, -0.4661218856307963, 3.741118141014307, 0.10480039037492729], [70, 1862, 1399.2722750017035, -286.0307434276999, -0.40882785793401316, 3.452370266800767, 0.09735620591583363], [60, 1804, 1515.6689728755591, -266.20066087454626, -0.3512649076262408, 3.1407287004442233, 0.08922141421573956], [59, 1756, 1619.6537389776506, -244.7605327494802, -0.30223809800726503, 2.851569139512236, 0.08224072597942804], [64, 1693, 1709.8520717742715, -220.603035778466, -0.2580375687699716, 2.5691933144059904, 0.07597578456037091], [52, 1635, 1787.62550343858, -199.92601566869484, -0.22367773930739737, 2.3083033859706235, 0.07043106324929338], [56, 1584, 1856.274143641366, -178.54679382067667, -0.192371147798708, 2.035700023968529, 0.0649070550053875], [74, 1532, 1912.3696124229427, -157.34880469258368, -0.16455898867084087, 1.7862237999055746, 0.05990155412094109], [114, 1461, 1947.9050194228269, -134.75242823321406, -0.13835626161396905, 1.5333825982636828, 0.054991738316519066], [131, 1398, 1954.9580709104546, -114.6680609492478, -0.1173099951917077, 1.2772216523868973, 0.05009801101711326], [441, 1338, 1920.0845665614802, -98.906396787632, -0.10302295899888957, 1.0918463199804667, 0.046738702326131774]], 'totalCount': 4961, 'weightedCount': 89251.24831711962, 'minAxis': 0, 'maxAxis': 1}}, '_className': 'DefaultDispatchConfiguration', '_version': 1}, 'isEditable': True, '_className': 'CompositeDecisionTool', '_version': 1}}, 'selectedConfiguration': 'CompositeDecisionTool2', 'indicatorsShown': ['Performance', 'Impact', 'Decision Cost', 'human volume', 'ml volume'], 'status': 'Active', '_className': 'Scenario', '_version': 1}}, 'selectedScenario': 'Scenario0', '_className': 'DefaultTaskModel', '_version': 1}
    
    # Function used to prcoess the input values from the user and preparing it as a payload for the model_decision.    
    def get_payload(input_values):
        result = {"fields": [], "values": [[]]}        
        for attribute in input_values[0]:
            result["fields"].append(attribute)            
        for value in input_values[1]:
            result["values"][0].append(value)            
        return result
    
    # Function for creating a case source for the dispatch    
    def create_case(input_values):        
        case = {
            "case": {
                "caseId": "CaseDataModel1"
            }
        }                
        for attribute in range(len(input_values[0])):
            case["case"][input_values[0][attribute]] = input_values[1][attribute]                        
        return case
    
    def model_decision(input_values):        
        result = get_payload(input_values)        
        header = {"Content-Type": "application/json", "Authorization": "Bearer " + token}        
        payload_scoring= {"input_data": [
                {
                        "fields": result["fields"],
                        "values": result["values"]
                        }
                  ]
        }                
        response_scoring = requests.post(input_values[2], json=payload_scoring, headers=header,verify=False)        
        case = create_case(input_values)        
        case["decisionTools"] = {
              "ml": {
                "outcome": json.loads(response_scoring.text)['predictions'][0]['values'][0][0],
                "confidence": json.loads(response_scoring.text)['predictions'][0]['values'][0][1][0]
              }
            }
        return case
    
    # Function for loading the model from user and passing the case source. pass json -> deserialize -> dispatch.            
    def dispatch(model, case):        
        load_model = aidc.load_task_model(model)        
        dispatch_result = aidc.dispatch(load_model, case)        
        return dispatch_result
    
    # Case -> dispatch using default model
    def dispatch_with_default(case):    
        load_model = aidc.load_task_model(default_model) 
        dispatch_result = aidc.dispatch(load_model, case)        
        return dispatch_result
        
    def score( payload ):                
        if len(payload["input_data"][0]["values"][0]) == 2: #json uploaded
            ml_results = model_decision(payload["input_data"][0]["values"][0][1])
            model_json=payload["input_data"][0]["values"][0][0]
            dispatch_result = dispatch(model_json, ml_results)            
            data = dispatch_result.valueOf()
        elif len(payload["input_data"][0]["values"][0]) == 1:
            ml_results = model_decision(payload["input_data"][0]["values"][0][0])            
            dispatch_result = dispatch_with_default(ml_results)            
            data = dispatch_result.valueOf()
        else:
            data = "Missing data"        
        # Score using the pre-defined model
        response = {
            'predictions': [{'fields': ['AIDC Decision'], 
                             'values': [[data]]}]
        } 
        return response
    
    return score

In [11]:
#Test custom function
test_values_for_ml = [["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings", "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"], 
                  ["no_checking", 8, "prior_payments_delayed", "repairs", 3094, "500_to_1000", "greater_7", 3, "male", "none", 3, "savings_insurance", 39, "none", "own", 1, "skilled", 1, "yes", "yes"],
                   model_serving_url]

test_values_for_human = [["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings", "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"], 
                  ["no_checking", 33, "prior_payments_delayed", "repairs", 3094, "500_to_1000", "greater_7", 3, "male", "none", 3, "savings_insurance", 39, "none", "own", 1, "skilled", 1, "yes", "yes"],
                   model_serving_url]

input_data = { "input_data": [{ "fields": [ "message" ],
                                "values": [[model_json, test_values_for_ml]]}]}    
function_result = my_deployable_function()( input_data )
print( function_result )

input_data = { "input_data": [{ "fields": [ "message" ],
                                "values": [[model_json,test_values_for_human]]}]}    
function_result = my_deployable_function()( input_data )
print( function_result )

input_data = { "input_data": [{ "fields": [ "message" ],
                                "values": [[test_values_for_human]]}]}    
function_result = my_deployable_function()( input_data )
print( function_result )

{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'ml', 'type': 'ml', 'confidence': 0.8580362993842072, 'outcome': 'No Risk'}]]}]}
{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'human', 'type': 'h'}]]}]}
{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'human', 'type': 'h'}]]}]}


In [12]:
#Prepare and deploy the custom function

#Find the AIDC custom software definition
file_path_AIDC = '/project_data/data_asset/aidc-1.3.zip'
specs_list = client.software_specifications.get_details()
for spec in specs_list["resources"]:
    name=(spec['metadata']['name'])
    if("aidc-spec" in name):
        print(name)
        base_software_spec_id =  client.software_specifications.get_id_by_name(name)


# Package extension metadata
meta_prop_pkg_ext = {
     client.package_extensions.ConfigurationMetaNames.NAME: "AIDC_custom_package",
     client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "AIDC custom package",
     client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
}
package_ext_details = client.package_extensions.store(meta_props=meta_prop_pkg_ext, file_path = file_path_AIDC)
package_ext_uid = client.package_extensions.get_uid(package_ext_details)

# Prepare software specificaion
meta_prop_sw_spec = {
     client.software_specifications.ConfigurationMetaNames.NAME: "AIDC_software_spec",
     client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "AIDC software specification",
     client.software_specifications.ConfigurationMetaNames.PACKAGE_EXTENSIONS : [{
         "guid": package_ext_uid
     }],
     client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
         "guid": base_software_spec_id
     }
}
software_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
software_spec_uid = client.software_specifications.get_uid(software_spec_details)

#Function meta details
meta_props = {
         client.repository.FunctionMetaNames.NAME: 'AIDC',
         client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: software_spec_uid
}

#Prepare the function
function_artifact = client.repository.store_function(meta_props=meta_props, function=my_deployable_function)
function_uid = client.repository.get_function_id(function_artifact)

#Prepare hardware specs
hardware_spec_id = client.hardware_specifications.get_id_by_name('S')

#Prepare the deployment metadata
deploy_meta = {
     client.deployments.ConfigurationMetaNames.NAME: "AIDC_function",
     client.deployments.ConfigurationMetaNames.ONLINE: {},
     client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id}
}

#Deploy the function
deployment_details = client.deployments.create(function_uid, meta_props=deploy_meta)

aidc-spec
Creating package extensions
SUCCESS


#######################################################################################

Synchronous deployment creation for uid: '0e2b151b-61f6-4760-af3d-04dd8e314140' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
......................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='af53f618-b0d8-45c5-a26a-3578517ab205'
------------------------------------------------------------------------------------------------




In [13]:
#Test the deployed function
deployment_uid = deployment_details['metadata']['id']
   
job_payload = {
     client.deployments.ScoringMetaNames.INPUT_DATA: [{
         "fields": [ "message" ],
         "values": [[model_json, test_values_for_ml]]
     }]
 }

function_result = client.deployments.score(deployment_uid, job_payload)
print( function_result )

job_payload = {
     client.deployments.ScoringMetaNames.INPUT_DATA: [{
         "fields": [ "message" ],
         "values": [[model_json,test_values_for_human]]
     }]
 }

function_result = client.deployments.score(deployment_uid, job_payload)
print( function_result )

job_payload = {
     client.deployments.ScoringMetaNames.INPUT_DATA: [{
         "fields": [ "message" ],
         "values": [[test_values_for_human]]
     }]
 }

function_result = client.deployments.score(deployment_uid, job_payload)
print( function_result )

{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'ml', 'type': 'ml', 'confidence': 0.8580362993842072, 'outcome': 'No Risk'}]]}]}
{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'human', 'type': 'h'}]]}]}
{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'human', 'type': 'h'}]]}]}
